In [2]:
class HeadingNode:
    """
    Represents a single heading node in the tree.
    """
    def __init__(
        self,
        node_id: int,
        parent_id: int,
        depth: int,
        text: str,
        page_number: int,
        is_capitalized: bool,
        is_bold: bool,
        is_italic: bool
    ):
        self.id = node_id
        self.parent_id = parent_id
        self.depth = depth
        self.text = text
        self.page_number = page_number
        self.is_capitalized = is_capitalized
        self.is_bold = is_bold
        self.is_italic = is_italic


class HeadingTree:
    """
    A simple tree structure that assigns parent nodes automatically
    based on depth. 
    - The root node is depth=0, id=0.
    - next_id increments with each added node.
    """
    def __init__(
        self,
        root_text: str,
        root_page_number: int,
        root_is_capitalized: bool = True,
        root_is_bold: bool = True,
        root_is_italic: bool = False
    ):
        self.next_id = 0
        self.nodes = {}  # key: node_id -> value: HeadingNode

        # Create the root node
        root_node = HeadingNode(
            node_id=self.next_id,
            parent_id=-1,  # or None, indicating no parent
            depth=0,
            text=root_text,
            page_number=root_page_number,
            is_capitalized=root_is_capitalized,
            is_bold=root_is_bold,
            is_italic=root_is_italic
        )
        self.nodes[self.next_id] = root_node
        self.next_id += 1

    def add_node(
        self,
        depth: int,
        text: str,
        page_number: int,
        is_capitalized: bool = False,
        is_bold: bool = False,
        is_italic: bool = False
    ) -> int:
        """
        Adds a new node at the specified depth. Automatically determines
        the parent_id as the node with the highest ID at (depth - 1).
        Returns the newly created node's ID.
        """
        if depth < 0:
            raise ValueError(f"Depth cannot be negative (got {depth}).")

        # If depth=0, it's another root-level node, but you said there's only one root. 
        # If you allow multiple nodes at depth=0, you'd handle that scenario here.
        if depth == 0:
            parent_id = -1  # or None
        else:
            parent_id = self._find_parent_for_depth(depth - 1)

        new_node = HeadingNode(
            node_id=self.next_id,
            parent_id=parent_id,
            depth=depth,
            text=text,
            page_number=page_number,
            is_capitalized=is_capitalized,
            is_bold=is_bold,
            is_italic=is_italic
        )
        self.nodes[self.next_id] = new_node
        self.next_id += 1

        return new_node.id

    def _find_parent_for_depth(self, parent_depth: int) -> int:
        """
        Finds the node with the highest ID among those whose depth == parent_depth.
        Returns its ID, or raises an error if none found.
        """
        if parent_depth < 0:
            # Means we're adding a root-level node (depth=0).
            return -1  # or None, as a sentinel
        # Filter nodes by the desired depth
        candidates = [n for n in self.nodes.values() if n.depth == parent_depth]
        if not candidates:
            raise ValueError(f"No existing node with depth {parent_depth} found.")
        # Return the node with the largest ID
        max_id_node = max(candidates, key=lambda x: x.id)
        return max_id_node.id

    def find_last_node(self) -> HeadingNode:
        """
        Returns the node with the highest ID.
        """
        if not self.nodes:
            raise ValueError("No nodes in the tree.")
        max_id_node = max(self.nodes.values(), key=lambda x: x.id)
        return max_id_node

    def show_tree(self):
        """
        Prints a text-based representation of the tree, 
        indenting by depth. Only shows the `text` field.
        """
        # One simple way: 
        #   1) Find all nodes sorted by (depth, id) 
        #   2) Print them in ascending order
        sorted_nodes = sorted(self.nodes.values(), key=lambda x: (x.id))
        tree_str = ""
        for node in sorted_nodes:
            indent = "  " * node.depth
            s = (f"Depth {node.depth} {indent}-> {node.text}")
            tree_str += s + "\n"\
        
        return tree_str
        

    def path_to_last_node(self):
        """
        Returns a list of HeadingNode objects forming the path 
        from the root node to the node with the highest ID.
        """
        last_node = self.find_last_node()
        path = self._path_to_node(last_node.id)

        answer = ''
        for node in path:
            answer += (f"Depth {node.depth} -> '{node.text}'\n")
        return answer

    def _path_to_node(self, node_id: int):
        """
        Internal method: build a path from the root to the specified node.
        """
        path = []
        current = self.nodes[node_id]  # start at the target node
        while current is not None and current.parent_id != -1:
            path.append(current)
            parent_id = current.parent_id
            current = self.nodes[parent_id] if parent_id in self.nodes else None

        # Add the root if it’s not already in the path
        if current is not None:
            path.append(current)

        # Reverse so root is at the front
        path.reverse()
        return path
    
    def show_formatted_tree_in_notebook(self):
        """
        Displays the tree in a Jupyter notebook cell with 
        capitalized, bold, or italic formatting via Markdown.
        Indentation is done by repeating spaces for each node's depth.
        """
        from IPython.display import display, Markdown
        
        # Sort the nodes by (depth, id) so we print in ascending depth order
        sorted_nodes = sorted(self.nodes.values(), key=lambda x: (x.id))
        markdown_lines = []

        for node in sorted_nodes:
            # Convert text according to metadata:
            #  1) Uppercase if is_capitalized
            #  2) Wrap in ** for bold
            #  3) Wrap in * for italic
            text = node.text
            if node.is_capitalized:
                text = text.upper()
            if node.is_bold:
                text = f"**{text}**"
            if node.is_italic:
                text = f"*{text}*"
            
            # Indent according to depth
            indent = "    " * node.depth  # 4 spaces per depth
            line = f"{indent}- (ID {node.id}, depth={node.depth}) {text}"
            markdown_lines.append(line)
        
        # Display as Markdown in the notebook
        display(Markdown("\n".join(markdown_lines)))

    def show_tree_ascii(self):
        """
        Prints the tree in an ASCII branch style like:
        
        a
        ├── b
        │   ├── d
        │   └── e
        └── c
        
        where each letter corresponds to the node's text.
        Assumes the root is at id=0, and its children are connected by parent_id.
        """
        from collections import defaultdict
        
        # Build an adjacency list: parent_id -> list of child_ids
        adjacency = defaultdict(list)
        for node_id, node in self.nodes.items():
            if node.parent_id != -1:
                adjacency[node.parent_id].append(node_id)
        
        # Sort each parent's children by ID to ensure stable ordering
        for parent in adjacency:
            adjacency[parent].sort()
        lines = []

        def _build_subtree(current_id, prefix, is_last):
            connector = "└── " if is_last else "├── "
            line = prefix + connector + self.nodes[current_id].text
            lines.append(line)
            
            # Depending on whether this node is the last child, adjust the prefix
            new_prefix = prefix + ("    " if is_last else "│   ")
            
            children = adjacency[current_id]
            for i, child_id in enumerate(children):
                _build_subtree(child_id, new_prefix, i == len(children) - 1)

        # Check if root (id=0) is present
        if 0 not in self.nodes:
            return "No root found at id=0."
        
        # Append root node plainly, then recursively build children
        root = self.nodes[0]
        lines.append(root.text)  # root text as first line
        
        # Build subtree for children of root
        children_of_root = adjacency[0]
        for i, child_id in enumerate(children_of_root):
            _build_subtree(child_id, "", i == len(children_of_root) - 1)

        # Return the ASCII tree as a single string
        return "\n".join(lines)


    def get_max_depth(self) -> int:
        """
        Returns the maximum depth of the tree.
        """
        if not self.nodes:
            return 0
        return max(node.depth for node in self.nodes.values())



In [27]:
tree_00 = HeadingTree("**ITEM 7. MANAGEMENT’S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS**", 25)

tree_00.add_node(1, "Overview", 26, True, True, False)
tree_00.add_node(1, "RESULTS OF OPERATIONS", 27, True, True, False)
# general
tree_00.add_node(1, "General", 27, True, True, False)
# net revenue
tree_00.add_node(1, "Net Revenue", 27, True, True, False)
# research and development
tree_00.add_node(1, "Research and Development", 28, True, True, False)
# selling, general and administrative
tree_00.add_node(1, "Selling, General and Administrative", 28, True, True, False)
# AMORTIZATION OF INTANGIBLES
tree_00.add_node(1, "Amortization of Intangibles", 29, True, True, False)
# RESTRUCTURING AND OTHER CHARGES
tree_00.add_node(1, "Restructuring and Other Charges", 29, True, True, False)
# PROVISION FOR INCOME TAXES
tree_00.add_node(1, "Provision for Income Taxes", 29, True, True, False)
# LIQUIDITY AND CAPITAL RESOURCES
tree_00.add_node(1, "Liquidity and Capital Resources", 30, True, True, False)
# Cash Flow from Operating Activities:
tree_00.add_node(2, "Cash Flow from Operating Activities", 30, False, True, False)
# Cash Flow from Investing Activities:
tree_00.add_node(2, "Cash Flow from Investing Activities", 30, False, True, False)
#Cash Flow from Financing Activities:
tree_00.add_node(2, "Cash Flow from Financing Activities", 30, False, True, False)
# Liquidity:
tree_00.add_node(2, "Liquidity", 30)
# OFF-BALANCE SHEET ARRANGEMENTS
tree_00.add_node(1, "Off-Balance Sheet Arrangements", 31, True, True, False)
# CONTRACTUAL CASH FLOWS
tree_00.add_node(1, "Contractual Cash Flows", 31, True, True, False)
# CRITICAL ACCOUNTING ESTIMATES
tree_00.add_node(1, "Critical Accounting Estimates", 31, True, True, False)
# Revenue Recognition
tree_00.add_node(2, "Revenue Recognition", 32, False, False, True)
# Inventory Valuation
tree_00.add_node(2, "Inventory Valuation", 32, False, False, True)
# Impairment of Long-Lived Assets. 
tree_00.add_node(2, "Impairment of Long-Lived Assets", 32, False, False, True)
# Business Combinations.
tree_00.add_node(2, "Business Combinations", 32, False, False, True)
# Share-Based Compensation
tree_00.add_node(2, "Share-Based Compensation", 32, False, False, True)
# Loss Contingencies
tree_00.add_node(2, "Loss Contingencies", 32, False, False, True)
# Income Taxes
tree_00.add_node(2, "Income Taxes", 32, False, False, True)
# OTHER MATTERS
tree_00.add_node(1, "Other Matters", 33, True, True, False)



print(tree_00.show_tree_ascii())
tree_00.show_formatted_tree_in_notebook()

**ITEM 7. MANAGEMENT’S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS**
├── Overview
├── RESULTS OF OPERATIONS
├── General
├── Net Revenue
├── Research and Development
├── Selling, General and Administrative
├── Amortization of Intangibles
├── Restructuring and Other Charges
├── Provision for Income Taxes
├── Liquidity and Capital Resources
│   ├── Cash Flow from Operating Activities
│   ├── Cash Flow from Investing Activities
│   ├── Cash Flow from Financing Activities
│   └── Liquidity
├── Off-Balance Sheet Arrangements
├── Contractual Cash Flows
├── Critical Accounting Estimates
│   ├── Revenue Recognition
│   ├── Inventory Valuation
│   ├── Impairment of Long-Lived Assets
│   ├── Business Combinations
│   ├── Share-Based Compensation
│   ├── Loss Contingencies
│   └── Income Taxes
└── Other Matters


- (ID 0, depth=0) ****ITEM 7. MANAGEMENT’S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS****
    - (ID 1, depth=1) **OVERVIEW**
    - (ID 2, depth=1) **RESULTS OF OPERATIONS**
    - (ID 3, depth=1) **GENERAL**
    - (ID 4, depth=1) **NET REVENUE**
    - (ID 5, depth=1) **RESEARCH AND DEVELOPMENT**
    - (ID 6, depth=1) **SELLING, GENERAL AND ADMINISTRATIVE**
    - (ID 7, depth=1) **AMORTIZATION OF INTANGIBLES**
    - (ID 8, depth=1) **RESTRUCTURING AND OTHER CHARGES**
    - (ID 9, depth=1) **PROVISION FOR INCOME TAXES**
    - (ID 10, depth=1) **LIQUIDITY AND CAPITAL RESOURCES**
        - (ID 11, depth=2) **Cash Flow from Operating Activities**
        - (ID 12, depth=2) **Cash Flow from Investing Activities**
        - (ID 13, depth=2) **Cash Flow from Financing Activities**
        - (ID 14, depth=2) Liquidity
    - (ID 15, depth=1) **OFF-BALANCE SHEET ARRANGEMENTS**
    - (ID 16, depth=1) **CONTRACTUAL CASH FLOWS**
    - (ID 17, depth=1) **CRITICAL ACCOUNTING ESTIMATES**
        - (ID 18, depth=2) *Revenue Recognition*
        - (ID 19, depth=2) *Inventory Valuation*
        - (ID 20, depth=2) *Impairment of Long-Lived Assets*
        - (ID 21, depth=2) *Business Combinations*
        - (ID 22, depth=2) *Share-Based Compensation*
        - (ID 23, depth=2) *Loss Contingencies*
        - (ID 24, depth=2) *Income Taxes*
    - (ID 25, depth=1) **OTHER MATTERS**

In [4]:
tree_03 = HeadingTree("ITEM 1. BUSINESS", 25)

# Company Overview
tree_03.add_node(1, "Company Overview", 1, False, True, False)
# Business Segments and Products
tree_03.add_node(1, "Business Segments and Products", 1, False, True, False)
# Sales and Distribution
tree_03.add_node(1, "Sales and Distribution", 1, False, True, False)
# International Operations
tree_03.add_node(1, "International Operations", 2, False, True, False)
# Contractual Arrangements
tree_03.add_node(1, "Contractual Arrangements", 2, False, True, False)
# Raw Materials
tree_03.add_node(1, "Raw Materials", 2, False, True, False)
# Competition and Healthcare Cost Containment
tree_03.add_node(1, "Competition and Healthcare Cost Containment", 3, False, True, False)
# Intellectual Property
tree_03.add_node(1, "Intellectual Property", 3, False, True, False)
# Research and Development
tree_03.add_node(1, "Research and Development", 4, False, True, False)
# Quality Management
tree_03.add_node(1, "Quality Management", 4, False, True, False)
# Government Regulation
tree_03.add_node(1, "Government Regulation", 5, False, True, False)
# Employees
tree_03.add_node(1, "Employees", 5, False, True, False)
# Available Information
tree_03.add_node(1, "Available Information", 5, False, True, False)

print(tree_03.show_tree_ascii())
tree_03.show_formatted_tree_in_notebook()



ITEM 1. BUSINESS
├── Company Overview
├── Business Segments and Products
├── Sales and Distribution
├── International Operations
├── Contractual Arrangements
├── Raw Materials
├── Competition and Healthcare Cost Containment
├── Intellectual Property
├── Research and Development
├── Quality Management
├── Government Regulation
├── Employees
└── Available Information


- (ID 0, depth=0) **ITEM 1. BUSINESS**
    - (ID 1, depth=1) **Company Overview**
    - (ID 2, depth=1) **Business Segments and Products**
    - (ID 3, depth=1) **Sales and Distribution**
    - (ID 4, depth=1) **International Operations**
    - (ID 5, depth=1) **Contractual Arrangements**
    - (ID 6, depth=1) **Raw Materials**
    - (ID 7, depth=1) **Competition and Healthcare Cost Containment**
    - (ID 8, depth=1) **Intellectual Property**
    - (ID 9, depth=1) **Research and Development**
    - (ID 10, depth=1) **Quality Management**
    - (ID 11, depth=1) **Government Regulation**
    - (ID 12, depth=1) **Employees**
    - (ID 13, depth=1) **Available Information**

In [5]:
tree_06 = HeadingTree("ITEM 1A. RISK FATORS", 10)

# The Company is subject to risks related to its international operations.
tree_06.add_node(1, "The Company is subject to risks related to its international operations.", 10, False, True, False)
# The Company faces intense competition in its markets, which could lead to reduced net sales, profitability and cash flow.
tree_06.add_node(1, "The Company faces intense competition in its markets, which could lead to reduced net sales, profitability and cash flow.", 11, False, True, False)
# Unfavorable worldwide, regional and local economic conditions and financial market volatility may negatively impact the Company and consumers of its products, which would negatively affect the Company’s financial performance and liquidity.
tree_06.add_node(1, "Unfavorable worldwide, regional and local economic conditions and financial market volatility may negatively impact the Company and consumers of its products, which would negatively affect the Company’s financial performance and liquidity.", 11, False, True, False)
# Volatility and increases in the costs of raw materials, energy, transportation, labor and other necessary supplies or services could harm the Company’s profits and cash flow.
tree_06.add_node(1, "Volatility and increases in the costs of raw materials, energy, transportation, labor and other necessary supplies or services could harm the Company’s profits and cash flow.", 12, False, True, False)
#Sales growth objectives may be difficult to achieve, and price increases, market and category declines and changes to the Company’s product and geographic mix may adversely impact the Company’s financial results.
tree_06.add_node(1, "Sales growth objectives may be difficult to achieve, and price increases, market and category declines and changes to the Company’s product and geographic mix may adversely impact the Company’s financial results.", 12, False, True, False)
# Dependence on key customers could adversely affect the Company’s business, financial condition and results of operations.
tree_06.add_node(1, "Dependence on key customers could adversely affect the Company’s business, financial condition and results of operations.", 12, False, True, False)
# Government regulations could impose material costs.
tree_06.add_node(1, "Government regulations could impose material costs.", 13, False, True, False)
# Reliance on a limited base of suppliers may result in disruptions to the Company’s business. p14
tree_06.add_node(1, "Reliance on a limited base of suppliers may result in disruptions to the Company’s business.", 14, False, True, False)
# Profitability and cash flow could suffer if the Company is unable to successfully implement its strategies, generate anticipated cost savings and efficiencies or efficiently manage supply chain and manufacturing processes. p14
tree_06.add_node(1, "Profitability and cash flow could suffer if the Company is unable to successfully implement its strategies, generate anticipated cost savings and efficiencies or efficiently manage supply chain and manufacturing processes.", 14, False, True, False)
# Product liability, commercial claims or other legal proceedings could adversely affect the Company’s sales and operating results, including cash flow. 
tree_06.add_node(1, "Product liability, commercial claims or other legal proceedings could adversely affect the Company’s sales and operating results, including cash flow.", 14, False, True, False)
# The Company may not successfully develop and introduce new products and line extensions or successfully expand into adjacent categories and countries, which could adversely impact its financial results. p 15
tree_06.add_node(1, "The Company may not successfully develop and introduce new products and line extensions or successfully expand into adjacent categories and countries, which could adversely impact its financial results.", 15, False, True, False)
# Acquisitions, new venture investments and divestitures may not be successful, which could impact the Company’s results.
tree_06.add_node(1, "Acquisitions, new venture investments and divestitures may not be successful, which could impact the Company’s results.", 15, False, True, False)
# Cyber attacks, privacy breaches or a failure of key information technology systems could adversely impact the Company’s ability to conduct business. p16
tree_06.add_node(1, "Cyber attacks, privacy breaches or a failure of key information technology systems could adversely impact the Company’s ability to conduct business.", 16, False, True, False)
# Loss of, or inability to attract, key personnel could adversely impact the Company’s business.
tree_06.add_node(1, "Loss of, or inability to attract, key personnel could adversely impact the Company’s business.", 16, False, True, False)
# Harm to the Company’s reputation or the reputation of one or more of its leading brands could have an adverse effect on the business.
tree_06.add_node(1, "Harm to the Company’s reputation or the reputation of one or more of its leading brands could have an adverse effect on the business.", 16, False, True, False)
# Environmental matters create potential liabilities.
tree_06.add_node(1, "Environmental matters create potential liabilities.", 16, False, True, False)
# The facilities of the Company and its suppliers are subject to disruption by events beyond the Company’s control. p17
tree_06.add_node(1, "The facilities of the Company and its suppliers are subject to disruption by events beyond the Company’s control.", 17, False, True, False)
# Failure to maximize, successfully assert or successfully defend the Company’s intellectual property rights could impact its competitiveness.
tree_06.add_node(1, "Failure to maximize, successfully assert or successfully defend the Company’s intellectual property rights could impact its competitiveness.", 17, False, True, False)
# If the Company is found to have infringed the intellectual property rights of others or cannot obtain necessary intellectual property rights from others, its competitiveness could be negatively impacted.
tree_06.add_node(1, "If the Company is found to have infringed the intellectual property rights of others or cannot obtain necessary intellectual property rights from others, its competitiveness could be negatively impacted.", 17, False, True, False)
# The Company’s substantial indebtedness could adversely affect its operations and financial results and prevent the Company from fulfilling its obligations, and the Company may incur substantially more debt in the future, which could exacerbate these risks. p 18
tree_06.add_node(1, "The Company’s substantial indebtedness could adversely affect its operations and financial results and prevent the Company from fulfilling its obligations, and the Company may incur substantially more debt in the future, which could exacerbate these risks.", 18, False, True, False)
# The Company’s continued growth and expansion and reliance on third-party service providers could adversely affect its internal control over financial reporting, which could harm its business and financial results.
tree_06.add_node(1, "The Company’s continued growth and expansion and reliance on third-party service providers could adversely affect its internal control over financial reporting, which could harm its business and financial results.", 18, False, True, False)
# The Company’s judgments regarding the accounting for tax positions and the resolution of tax disputes may impact the Company’s earnings and cash flow. p19
tree_06.add_node(1, "The Company’s judgments regarding the accounting for tax positions and the resolution of tax disputes may impact the Company’s earnings and cash flow.", 19, False, True, False)
# The estimates and assumptions on which the Company’s financial statement projections are based may prove to be inaccurate, which may cause its actual results to materially differ from such projections, which may adversely affect the Company’s future cash flows and stock price.
tree_06.add_node(1, "The estimates and assumptions on which the Company’s financial statement projections are based may prove to be inaccurate, which may cause its actual results to materially differ from such projections, which may adversely affect the Company’s future cash flows and stock price.", 19, False, True, False)
# The Company may not continue to pay dividends or repurchase its stock.
tree_06.add_node(1, "The Company may not continue to pay dividends or repurchase its stock.", 19, False, True, False)
# The Company’s business could be negatively impacted as a result of stockholder activism or an unsolicited takeover proposal or a proxy contest.
tree_06.add_node(1, "The Company’s business could be negatively impacted as a result of stockholder activism or an unsolicited takeover proposal or a proxy contest.", 19, False, True, False)

print(tree_06.show_tree_ascii())
tree_06.show_formatted_tree_in_notebook()



ITEM 1A. RISK FATORS
├── The Company is subject to risks related to its international operations.
├── The Company faces intense competition in its markets, which could lead to reduced net sales, profitability and cash flow.
├── Unfavorable worldwide, regional and local economic conditions and financial market volatility may negatively impact the Company and consumers of its products, which would negatively affect the Company’s financial performance and liquidity.
├── Volatility and increases in the costs of raw materials, energy, transportation, labor and other necessary supplies or services could harm the Company’s profits and cash flow.
├── Sales growth objectives may be difficult to achieve, and price increases, market and category declines and changes to the Company’s product and geographic mix may adversely impact the Company’s financial results.
├── Dependence on key customers could adversely affect the Company’s business, financial condition and results of operations.
├── Govern

- (ID 0, depth=0) **ITEM 1A. RISK FATORS**
    - (ID 1, depth=1) **The Company is subject to risks related to its international operations.**
    - (ID 2, depth=1) **The Company faces intense competition in its markets, which could lead to reduced net sales, profitability and cash flow.**
    - (ID 3, depth=1) **Unfavorable worldwide, regional and local economic conditions and financial market volatility may negatively impact the Company and consumers of its products, which would negatively affect the Company’s financial performance and liquidity.**
    - (ID 4, depth=1) **Volatility and increases in the costs of raw materials, energy, transportation, labor and other necessary supplies or services could harm the Company’s profits and cash flow.**
    - (ID 5, depth=1) **Sales growth objectives may be difficult to achieve, and price increases, market and category declines and changes to the Company’s product and geographic mix may adversely impact the Company’s financial results.**
    - (ID 6, depth=1) **Dependence on key customers could adversely affect the Company’s business, financial condition and results of operations.**
    - (ID 7, depth=1) **Government regulations could impose material costs.**
    - (ID 8, depth=1) **Reliance on a limited base of suppliers may result in disruptions to the Company’s business.**
    - (ID 9, depth=1) **Profitability and cash flow could suffer if the Company is unable to successfully implement its strategies, generate anticipated cost savings and efficiencies or efficiently manage supply chain and manufacturing processes.**
    - (ID 10, depth=1) **Product liability, commercial claims or other legal proceedings could adversely affect the Company’s sales and operating results, including cash flow.**
    - (ID 11, depth=1) **The Company may not successfully develop and introduce new products and line extensions or successfully expand into adjacent categories and countries, which could adversely impact its financial results.**
    - (ID 12, depth=1) **Acquisitions, new venture investments and divestitures may not be successful, which could impact the Company’s results.**
    - (ID 13, depth=1) **Cyber attacks, privacy breaches or a failure of key information technology systems could adversely impact the Company’s ability to conduct business.**
    - (ID 14, depth=1) **Loss of, or inability to attract, key personnel could adversely impact the Company’s business.**
    - (ID 15, depth=1) **Harm to the Company’s reputation or the reputation of one or more of its leading brands could have an adverse effect on the business.**
    - (ID 16, depth=1) **Environmental matters create potential liabilities.**
    - (ID 17, depth=1) **The facilities of the Company and its suppliers are subject to disruption by events beyond the Company’s control.**
    - (ID 18, depth=1) **Failure to maximize, successfully assert or successfully defend the Company’s intellectual property rights could impact its competitiveness.**
    - (ID 19, depth=1) **If the Company is found to have infringed the intellectual property rights of others or cannot obtain necessary intellectual property rights from others, its competitiveness could be negatively impacted.**
    - (ID 20, depth=1) **The Company’s substantial indebtedness could adversely affect its operations and financial results and prevent the Company from fulfilling its obligations, and the Company may incur substantially more debt in the future, which could exacerbate these risks.**
    - (ID 21, depth=1) **The Company’s continued growth and expansion and reliance on third-party service providers could adversely affect its internal control over financial reporting, which could harm its business and financial results.**
    - (ID 22, depth=1) **The Company’s judgments regarding the accounting for tax positions and the resolution of tax disputes may impact the Company’s earnings and cash flow.**
    - (ID 23, depth=1) **The estimates and assumptions on which the Company’s financial statement projections are based may prove to be inaccurate, which may cause its actual results to materially differ from such projections, which may adversely affect the Company’s future cash flows and stock price.**
    - (ID 24, depth=1) **The Company may not continue to pay dividends or repurchase its stock.**
    - (ID 25, depth=1) **The Company’s business could be negatively impacted as a result of stockholder activism or an unsolicited takeover proposal or a proxy contest.**

In [7]:
tree_07 = HeadingTree("ITEM 1. BUSINESS", 6)
# Background
tree_07.add_node(1, "Background", 6, False, True, False)
# Revitalization Plan
tree_07.add_node(2, "Revitalization Plan", 6, False, True, True)
# Industry Overview p7
tree_07.add_node(1, "Industry Overview", 7, False, True, False)
# Global Competitors' Market Capitalization
tree_07.add_node(2, "Global Competitors' Market Capitalization", 7, False, True, True)
# Our Products
tree_07.add_node(2, "Our Products", 7, False, True, True)
# Brands sold in the U.S. p8
tree_07.add_node(2, "Brands sold in the U.S.", 8, False, True, True)
# Brands sold in Canada p9
tree_07.add_node(2, "Brands sold in Canada", 9, False, True, True)
# Brands sold in International p10
tree_07.add_node(2, "Brands sold in International", 10, False, True, True)
# Our Segments
tree_07.add_node(1, "Our Segments", 10, False, True, False)
# United States Segment
tree_07.add_node(2, "United States Segment", 10, False, True, True)
# Sales and Distribution
tree_07.add_node(3, "Sales and Distribution", 10, False, False, True)
# Channels
tree_07.add_node(4, "Channels", 10, False, False, True)
# Manufacturing, Production and Packaging p11
tree_07.add_node(3, "Manufacturing, Production and Packaging", 11, False, False, True)
# Brewing Raw Materials
tree_07.add_node(4, "Brewing Raw Materials", 11, False, False, True)
# Packaging Materials
tree_07.add_node(4, "Packaging Materials", 11, False, False, True)
# Contract Manufacturing p12
tree_07.add_node(4, "Contract Manufacturing", 12, False, False, True)
# Seasonality of the Business
tree_07.add_node(3, "Seasonality of the Business", 12, False, False, True)
# Known Trends and Competitive Conditions
tree_07.add_node(3, "Known Trends and Competitive Conditions", 12, False, False, True)
# 2019 U.S. Beer Industry Overview
tree_07.add_node(4, "2019 U.S. Beer Industry Overview", 12, False, False, True)
# Our Competitive Position
tree_07.add_node(4, "Our Competitive Position", 13, False, False, True)
# Reculation p13
tree_07.add_node(3, "Regulation", 13, False, False, True)
# Canada Segment
tree_07.add_node(2, "Canada Segment", 13, False, True, True)
# Sales and Distribution
tree_07.add_node(3, "Sales and Distribution", 13, False, False, True)
# Channels
tree_07.add_node(4, "Channels", 13, False, False, True)
# Province of Ontario -14
tree_07.add_node(4, "Province of Ontario", 14, False, False, True)
# Province of Québec
tree_07.add_node(4, "Province of Québec", 14, False, False, True)
# Province of British Columbia
tree_07.add_node(4, "Province of British Columbia", 14, False, False, True)
# Province of Alberta
tree_07.add_node(4, "Province of Alberta", 14, False, False, True)
# Other Provinces
tree_07.add_node(4, "Other Provinces", 14, False, False, True)
# Manufacturing, Production and Packaging p15
tree_07.add_node(3, "Manufacturing, Production and Packaging", 15, False, False, True)
# Brewing Raw Materials
tree_07.add_node(4, "Brewing Raw Materials", 15, False, False, True)
# Packaging Materials
tree_07.add_node(4, "Packaging Materials", 15, False, False, True)
# Contract Manufacturing
tree_07.add_node(4, "Contract Manufacturing", 15, False, False, True)
# Seasonality of Business p16
tree_07.add_node(3, "Seasonality of Business", 16, False, False, True)
# Known Trends and Competitive Conditions
tree_07.add_node(3, "Known Trends and Competitive Conditions", 16, False, False, True)
# 2019 Canada Beer Industry Overview
tree_07.add_node(4, "2019 Canada Beer Industry Overview", 16, False, False, True)
# Regulation
tree_07.add_node(3, "Regulation", 16, False, False, True)
# Europe Segment
tree_07.add_node(2, "Europe Segment", 17, False, True, True)
# Sales and Distribution
tree_07.add_node(3, "Sales and Distribution", 17, False, False, True)
# Channels
tree_07.add_node(4, "Channels", 17, False, False, True)
# Manufacturing, Production and Packaging
tree_07.add_node(3, "Manufacturing, Production and Packaging", 17, False, False, True)
# Brewing Raw Materials
tree_07.add_node(4, "Brewing Raw Materials", 17, False, False, True)
# Packaging Materials p18
tree_07.add_node(4, "Packaging Materials", 18, False, False, True)
# Seasonality of Business
tree_07.add_node(3, "Seasonality of Business", 18, False, False, True)
# Known Trends and Competitive Conditions
tree_07.add_node(3, "Known Trends and Competitive Conditions", 18, False, False, True)
# 2019 Europe Beer Industry Overview 
tree_07.add_node(4, "2019 Europe Beer Industry Overview", 18, False, False, True)
# Regulation p19
tree_07.add_node(3, "Regulation", 19, False, False, True)
# International Segment
tree_07.add_node(2, "International Segment", 19, False, True, True)
# Latin America p20
tree_07.add_node(3, "Latin America", 20, False, False, True)
# Asia Pacific and Africa
tree_07.add_node(3, "Asia Pacific and Africa", 20, False, False, True)
# Corporate
tree_07.add_node(2, "Corporate", 20, False, True, True)
# Other Information 
tree_07.add_node(1, "Other Information", 20, False, True, False)
# Global Intellectual Property
tree_07.add_node(2, "Global Intellectual Property", 20, False, True, True)
# Sustainability
tree_07.add_node(2, "Sustainability", 21, False, True, True)
# Environmental Matters p21
tree_07.add_node(1, "Environmental Matters", 21, False, False, True)
# Employees
tree_07.add_node(1, "Employees", 21, False, False, True)
# Available Information
tree_07.add_node(1, "Available Information", 21, False, False, True)
# Information About Our Executive Officers p22
tree_07.add_node(1, "Information About Our Executive Officers", 22, False, False, True)

print(tree_07.show_tree_ascii())
tree_07.show_formatted_tree_in_notebook()




ITEM 1. BUSINESS
├── Background
│   └── Revitalization Plan
├── Industry Overview
│   ├── Global Competitors' Market Capitalization
│   ├── Our Products
│   ├── Brands sold in the U.S.
│   ├── Brands sold in Canada
│   └── Brands sold in International
├── Our Segments
│   ├── United States Segment
│   │   ├── Sales and Distribution
│   │   │   └── Channels
│   │   ├── Manufacturing, Production and Packaging
│   │   │   ├── Brewing Raw Materials
│   │   │   ├── Packaging Materials
│   │   │   └── Contract Manufacturing
│   │   ├── Seasonality of the Business
│   │   ├── Known Trends and Competitive Conditions
│   │   │   ├── 2019 U.S. Beer Industry Overview
│   │   │   └── Our Competitive Position
│   │   └── Regulation
│   ├── Canada Segment
│   │   ├── Sales and Distribution
│   │   │   ├── Channels
│   │   │   ├── Province of Ontario
│   │   │   ├── Province of Québec
│   │   │   ├── Province of British Columbia
│   │   │   ├── Province of Alberta
│   │   │   └── Other Provinces
│   

- (ID 0, depth=0) **ITEM 1. BUSINESS**
    - (ID 1, depth=1) **Background**
        - (ID 2, depth=2) ***Revitalization Plan***
    - (ID 3, depth=1) **Industry Overview**
        - (ID 4, depth=2) ***Global Competitors' Market Capitalization***
        - (ID 5, depth=2) ***Our Products***
        - (ID 6, depth=2) ***Brands sold in the U.S.***
        - (ID 7, depth=2) ***Brands sold in Canada***
        - (ID 8, depth=2) ***Brands sold in International***
    - (ID 9, depth=1) **Our Segments**
        - (ID 10, depth=2) ***United States Segment***
            - (ID 11, depth=3) *Sales and Distribution*
                - (ID 12, depth=4) *Channels*
            - (ID 13, depth=3) *Manufacturing, Production and Packaging*
                - (ID 14, depth=4) *Brewing Raw Materials*
                - (ID 15, depth=4) *Packaging Materials*
                - (ID 16, depth=4) *Contract Manufacturing*
            - (ID 17, depth=3) *Seasonality of the Business*
            - (ID 18, depth=3) *Known Trends and Competitive Conditions*
                - (ID 19, depth=4) *2019 U.S. Beer Industry Overview*
                - (ID 20, depth=4) *Our Competitive Position*
            - (ID 21, depth=3) *Regulation*
        - (ID 22, depth=2) ***Canada Segment***
            - (ID 23, depth=3) *Sales and Distribution*
                - (ID 24, depth=4) *Channels*
                - (ID 25, depth=4) *Province of Ontario*
                - (ID 26, depth=4) *Province of Québec*
                - (ID 27, depth=4) *Province of British Columbia*
                - (ID 28, depth=4) *Province of Alberta*
                - (ID 29, depth=4) *Other Provinces*
            - (ID 30, depth=3) *Manufacturing, Production and Packaging*
                - (ID 31, depth=4) *Brewing Raw Materials*
                - (ID 32, depth=4) *Packaging Materials*
                - (ID 33, depth=4) *Contract Manufacturing*
            - (ID 34, depth=3) *Seasonality of Business*
            - (ID 35, depth=3) *Known Trends and Competitive Conditions*
                - (ID 36, depth=4) *2019 Canada Beer Industry Overview*
            - (ID 37, depth=3) *Regulation*
        - (ID 38, depth=2) ***Europe Segment***
            - (ID 39, depth=3) *Sales and Distribution*
                - (ID 40, depth=4) *Channels*
            - (ID 41, depth=3) *Manufacturing, Production and Packaging*
                - (ID 42, depth=4) *Brewing Raw Materials*
                - (ID 43, depth=4) *Packaging Materials*
            - (ID 44, depth=3) *Seasonality of Business*
            - (ID 45, depth=3) *Known Trends and Competitive Conditions*
                - (ID 46, depth=4) *2019 Europe Beer Industry Overview*
            - (ID 47, depth=3) *Regulation*
        - (ID 48, depth=2) ***International Segment***
            - (ID 49, depth=3) *Latin America*
            - (ID 50, depth=3) *Asia Pacific and Africa*
        - (ID 51, depth=2) ***Corporate***
    - (ID 52, depth=1) **Other Information**
        - (ID 53, depth=2) ***Global Intellectual Property***
        - (ID 54, depth=2) ***Sustainability***
    - (ID 55, depth=1) *Environmental Matters*
    - (ID 56, depth=1) *Employees*
    - (ID 57, depth=1) *Available Information*
    - (ID 58, depth=1) *Information About Our Executive Officers*

In [8]:
tree_09 = HeadingTree("ITEM 1. BUSINESS", 3)
# AUTOMATION SOLUTIONS
tree_09.add_node(1, "AUTOMATION SOLUTIONS", 3, True, True, False)
# Measurement & Analytical Instrumentation p4
tree_09.add_node(2, "Measurement & Analytical Instrumentation", 4, False, False, False)
# Valves, Actuators & Regulators
tree_09.add_node(2, "Valves, Actuators & Regulators", 4, False, False, False)
# Industrial Solutions
tree_09.add_node(2, "Industrial Solutions", 4, False, False, False)
# Process Control Systems & Solutions p5
tree_09.add_node(2, "Process Control Systems & Solutions", 5, False, False, False)
#  Plantweb Digital Ecosystem
tree_09.add_node(2, "Plantweb Digital Ecosystem", 5, False, False, False)
#  Industry Services
tree_09.add_node(2, "Industry Services", 5, False, False, False)
# Distribution 
tree_09.add_node(2, "Distribution", 6, False, False, False)
# Brands
tree_09.add_node(2, "Brands", 6, False, False, False)
#  COMMERCIAL & RESIDENTIAL SOLUTIONS p6
tree_09.add_node(1, "COMMERCIAL & RESIDENTIAL SOLUTIONS", 6, True, True, False)
#  CLIMATE TECHNOLOGIES
tree_09.add_node(1, "CLIMATE TECHNOLOGIES", 6, True, True, False)
# Residential and Commercial Heating and Air Conditioning
tree_09.add_node(2, "Residential and Commercial Heating and Air Conditioning", 6, False, False, False)
# Commercial and Industrial Refrigeration
tree_09.add_node(2, "Commercial and Industrial Refrigeration", 6, False, False, False)
# Services and Solutions
tree_09.add_node(2, "Services and Solutions", 6, False, False, False)
# Distribution
tree_09.add_node(2, "Distribution", 6, False, False, False)
# Brands p7
tree_09.add_node(2, "Brands", 7, False, False, False)
# TOOLS & HOME PRODUCTS
tree_09.add_node(1, "TOOLS & HOME PRODUCTS", 7, True, True, False)
# Professional and Do-It-Yourself Tools
tree_09.add_node(2, "Professional and Do-It-Yourself Tools", 7, False, False, False)
# Appliance Solutions
tree_09.add_node(2, "Appliance Solutions", 7, False, False, False)
# Distribution
tree_09.add_node(2, "Distribution", 7, False, False, False)
# Brands
tree_09.add_node(2, "Brands", 7, False, False, False)
#  DISCONTINUED OPERATIONS
tree_09.add_node(1, "DISCONTINUED OPERATIONS", 7, True, True, False)
#  PRODUCTION p8
tree_09.add_node(1, "PRODUCTION", 8, True, True, False)
# RAW MATERIALS
tree_09.add_node(1, "RAW MATERIALS", 8, True, True, False)
# PATENTS, TRADEMARKS AND LICENSES
tree_09.add_node(1, "PATENTS, TRADEMARKS AND LICENSES", 8, True, True, False)
# BACKLOG
tree_09.add_node(1, "BACKLOG", 8, True, True, False)
# COMPETITION
tree_09.add_node(1, "COMPETITION", 8, True, True, False)
# RESEARCH AND DEVELOPMENT
tree_09.add_node(1, "RESEARCH AND DEVELOPMENT", 8, True, True, False)
# ENVIRONMENT p9
tree_09.add_node(1, "ENVIRONMENT", 9, True, True, False)
# EMPLOYEES
tree_09.add_node(1, "EMPLOYEES", 9, True, True, False)
# DOMESTIC AND FOREIGN OPERATIONS
tree_09.add_node(1, "DOMESTIC AND FOREIGN OPERATIONS", 9, True, True, False)
# INTERNET ACCESS
tree_09.add_node(1, "INTERNET ACCESS", 9, True, True, False)

print(tree_09.show_tree_ascii())
tree_09.show_formatted_tree_in_notebook()






ITEM 1. BUSINESS
├── AUTOMATION SOLUTIONS
│   ├── Measurement & Analytical Instrumentation
│   ├── Valves, Actuators & Regulators
│   ├── Industrial Solutions
│   ├── Process Control Systems & Solutions
│   ├── Plantweb Digital Ecosystem
│   ├── Industry Services
│   ├── Distribution
│   └── Brands
├── COMMERCIAL & RESIDENTIAL SOLUTIONS
├── CLIMATE TECHNOLOGIES
│   ├── Residential and Commercial Heating and Air Conditioning
│   ├── Commercial and Industrial Refrigeration
│   ├── Services and Solutions
│   ├── Distribution
│   └── Brands
├── TOOLS & HOME PRODUCTS
│   ├── Professional and Do-It-Yourself Tools
│   ├── Appliance Solutions
│   ├── Distribution
│   └── Brands
├── DISCONTINUED OPERATIONS
├── PRODUCTION
├── RAW MATERIALS
├── PATENTS, TRADEMARKS AND LICENSES
├── BACKLOG
├── COMPETITION
├── RESEARCH AND DEVELOPMENT
├── ENVIRONMENT
├── EMPLOYEES
├── DOMESTIC AND FOREIGN OPERATIONS
└── INTERNET ACCESS


- (ID 0, depth=0) **ITEM 1. BUSINESS**
    - (ID 1, depth=1) **AUTOMATION SOLUTIONS**
        - (ID 2, depth=2) Measurement & Analytical Instrumentation
        - (ID 3, depth=2) Valves, Actuators & Regulators
        - (ID 4, depth=2) Industrial Solutions
        - (ID 5, depth=2) Process Control Systems & Solutions
        - (ID 6, depth=2) Plantweb Digital Ecosystem
        - (ID 7, depth=2) Industry Services
        - (ID 8, depth=2) Distribution
        - (ID 9, depth=2) Brands
    - (ID 10, depth=1) **COMMERCIAL & RESIDENTIAL SOLUTIONS**
    - (ID 11, depth=1) **CLIMATE TECHNOLOGIES**
        - (ID 12, depth=2) Residential and Commercial Heating and Air Conditioning
        - (ID 13, depth=2) Commercial and Industrial Refrigeration
        - (ID 14, depth=2) Services and Solutions
        - (ID 15, depth=2) Distribution
        - (ID 16, depth=2) Brands
    - (ID 17, depth=1) **TOOLS & HOME PRODUCTS**
        - (ID 18, depth=2) Professional and Do-It-Yourself Tools
        - (ID 19, depth=2) Appliance Solutions
        - (ID 20, depth=2) Distribution
        - (ID 21, depth=2) Brands
    - (ID 22, depth=1) **DISCONTINUED OPERATIONS**
    - (ID 23, depth=1) **PRODUCTION**
    - (ID 24, depth=1) **RAW MATERIALS**
    - (ID 25, depth=1) **PATENTS, TRADEMARKS AND LICENSES**
    - (ID 26, depth=1) **BACKLOG**
    - (ID 27, depth=1) **COMPETITION**
    - (ID 28, depth=1) **RESEARCH AND DEVELOPMENT**
    - (ID 29, depth=1) **ENVIRONMENT**
    - (ID 30, depth=1) **EMPLOYEES**
    - (ID 31, depth=1) **DOMESTIC AND FOREIGN OPERATIONS**
    - (ID 32, depth=1) **INTERNET ACCESS**

In [9]:
tree_10 = HeadingTree("ITEM 1. BUSINESS", 6)
# THE COMPANY
tree_10.add_node(1, "THE COMPANY", 6, True, True, False)
# Avaiable Information
tree_10.add_node(2, "Avaiable Information", 6, False, True, False)
# MERGER AND SEPARATION
tree_10.add_node(1, "MERGER AND SEPARATION", 6, True, True, False)
# About Dow
tree_10.add_node(2, "About Dow", 6, False, True, False)
# BUSINESS SEGMENTS AND PRODUCTS p7
tree_10.add_node(1, "BUSINESS SEGMENTS AND PRODUCTS", 7, True, True, False)
# PACKAGING & SPECIALTY PLASTICS
tree_10.add_node(1, "PACKAGING & SPECIALTY PLASTICS", 7, True, True, False)
# Hydrocarbons & Energy
tree_10.add_node(2, "Hydrocarbons & Energy", 7, False, False, True)
# Packaging and Specialty Plastics
tree_10.add_node(2, "Packaging and Specialty Plastics", 7, False, False, True)
# Divestiture
tree_10.add_node(2, "Divestiture", 7, False, True, False)
# Products p8
tree_10.add_node(2, "Products", 8, False, True, False)
# Joint Ventures p9
tree_10.add_node(2, "Joint Ventures", 9, False, True, False)
# Current and Future Investments
tree_10.add_node(2, "Current and Future Investments", 9, False, True, False)
# INDUSTRIAL INTERMEDIATES & INFRASTRUCTURE p10
tree_10.add_node(1, "INDUSTRIAL INTERMEDIATES & INFRASTRUCTURE", 10, True, True, False)
# Industrial Solutions
tree_10.add_node(2, "Industrial Solutions", 10, False, False, True)
# Polyurethanes & Construction Chemicals
tree_10.add_node(2, "Polyurethanes & Construction Chemicals", 10, False, False, True)
# Products p11
tree_10.add_node(2, "Products", 11, False, True, False)
# Joint Ventures p12
tree_10.add_node(2, "Joint Ventures", 12, False, True, False)
# Current and Future Investments
tree_10.add_node(2, "Current and Future Investments", 12, False, True, False)
# PERFORMANCE MATERIALS & COATINGS
tree_10.add_node(1, "PERFORMANCE MATERIALS & COATINGS", 12, True, True, False)
# Coatings & Performance Monomers
tree_10.add_node(2, "Coatings & Performance Monomers", 12, False, False, True)
# Consumer Solutions
tree_10.add_node(2, "Consumer Solutions", 12, False, False, True)
# Products p13
tree_10.add_node(2, "Products", 13, False, True, False)
# Current and Future Investments p14
tree_10.add_node(2, "Current and Future Investments", 14, False, True, False)
# CORPORATE
tree_10.add_node(1, "CORPORATE", 14, True, True, False)
# RAW MATERIALS
tree_10.add_node(1, "RAW MATERIALS", 14, True, True, False)
# INDUSTRY SEGMENTS AND GEOGRAPHIC REGION RESULTS
tree_10.add_node(1, "INDUSTRY SEGMENTS AND GEOGRAPHIC REGION RESULTS", 14, True, True, False)
# SIGNIFICANT CUSTOMERS AND PRODUCTS
tree_10.add_node(1, "SIGNIFICANT CUSTOMERS AND PRODUCTS", 14, True, True, False)
# PATENTS, LICENSES AND TRADEMARKS
tree_10.add_node(1, "PATENTS, LICENSES AND TRADEMARKS", 14, True, True, False)
# PRINCIPAL PARTLY OWNED COMPANIES p15
tree_10.add_node(1, "PRINCIPAL PARTLY OWNED COMPANIES", 15, True, True, False)
# PROTECTION OF THE ENVIRONMENT
tree_10.add_node(1, "PROTECTION OF THE ENVIRONMENT", 15, True, True, False)
# EMPLOYEES
tree_10.add_node(1, "EMPLOYEES", 15, True, True, False)
# OTHER ACTIVITIES
tree_10.add_node(1, "OTHER ACTIVITIES", 15, True, True, False)
# EXECUTIVE OFFICERS OF THE REGISTRANT p16
tree_10.add_node(1, "EXECUTIVE OFFICERS OF THE REGISTRANT", 16, True, True, False)

print(tree_10.show_tree_ascii())
tree_10.show_formatted_tree_in_notebook()








ITEM 1. BUSINESS
├── THE COMPANY
│   └── Avaiable Information
├── MERGER AND SEPARATION
│   └── About Dow
├── BUSINESS SEGMENTS AND PRODUCTS
├── PACKAGING & SPECIALTY PLASTICS
│   ├── Hydrocarbons & Energy
│   ├── Packaging and Specialty Plastics
│   ├── Divestiture
│   ├── Products
│   ├── Joint Ventures
│   └── Current and Future Investments
├── INDUSTRIAL INTERMEDIATES & INFRASTRUCTURE
│   ├── Industrial Solutions
│   ├── Polyurethanes & Construction Chemicals
│   ├── Products
│   ├── Joint Ventures
│   └── Current and Future Investments
├── PERFORMANCE MATERIALS & COATINGS
│   ├── Coatings & Performance Monomers
│   ├── Consumer Solutions
│   ├── Products
│   └── Current and Future Investments
├── CORPORATE
├── RAW MATERIALS
├── INDUSTRY SEGMENTS AND GEOGRAPHIC REGION RESULTS
├── SIGNIFICANT CUSTOMERS AND PRODUCTS
├── PATENTS, LICENSES AND TRADEMARKS
├── PRINCIPAL PARTLY OWNED COMPANIES
├── PROTECTION OF THE ENVIRONMENT
├── EMPLOYEES
├── OTHER ACTIVITIES
└── EXECUTIVE OFFICERS OF T

- (ID 0, depth=0) **ITEM 1. BUSINESS**
    - (ID 1, depth=1) **THE COMPANY**
        - (ID 2, depth=2) **Avaiable Information**
    - (ID 3, depth=1) **MERGER AND SEPARATION**
        - (ID 4, depth=2) **About Dow**
    - (ID 5, depth=1) **BUSINESS SEGMENTS AND PRODUCTS**
    - (ID 6, depth=1) **PACKAGING & SPECIALTY PLASTICS**
        - (ID 7, depth=2) *Hydrocarbons & Energy*
        - (ID 8, depth=2) *Packaging and Specialty Plastics*
        - (ID 9, depth=2) **Divestiture**
        - (ID 10, depth=2) **Products**
        - (ID 11, depth=2) **Joint Ventures**
        - (ID 12, depth=2) **Current and Future Investments**
    - (ID 13, depth=1) **INDUSTRIAL INTERMEDIATES & INFRASTRUCTURE**
        - (ID 14, depth=2) *Industrial Solutions*
        - (ID 15, depth=2) *Polyurethanes & Construction Chemicals*
        - (ID 16, depth=2) **Products**
        - (ID 17, depth=2) **Joint Ventures**
        - (ID 18, depth=2) **Current and Future Investments**
    - (ID 19, depth=1) **PERFORMANCE MATERIALS & COATINGS**
        - (ID 20, depth=2) *Coatings & Performance Monomers*
        - (ID 21, depth=2) *Consumer Solutions*
        - (ID 22, depth=2) **Products**
        - (ID 23, depth=2) **Current and Future Investments**
    - (ID 24, depth=1) **CORPORATE**
    - (ID 25, depth=1) **RAW MATERIALS**
    - (ID 26, depth=1) **INDUSTRY SEGMENTS AND GEOGRAPHIC REGION RESULTS**
    - (ID 27, depth=1) **SIGNIFICANT CUSTOMERS AND PRODUCTS**
    - (ID 28, depth=1) **PATENTS, LICENSES AND TRADEMARKS**
    - (ID 29, depth=1) **PRINCIPAL PARTLY OWNED COMPANIES**
    - (ID 30, depth=1) **PROTECTION OF THE ENVIRONMENT**
    - (ID 31, depth=1) **EMPLOYEES**
    - (ID 32, depth=1) **OTHER ACTIVITIES**
    - (ID 33, depth=1) **EXECUTIVE OFFICERS OF THE REGISTRANT**

In [10]:
tree_15 = HeadingTree("ITEM 1A. RISK FACTORS", 18)
# Construction and maintenance sites are inherently dangerous workplaces. If we fail to maintain safe work sites, we can be exposed to significant financial losses as well as civil and criminal liabilities.
tree_15.add_node(1, "Construction and maintenance sites are inherently dangerous workplaces. If we fail to maintain safe work sites, we can be exposed to significant financial losses as well as civil and criminal liabilities.", 18, False, True, True)
# Our project execution activities may result in liability for faulty engineering services.
tree_15.add_node(1, "Our project execution activities may result in liability for faulty engineering services.", 18, False, True, True)
# The outcome of pending and future claims and litigation could have a material adverse impact on our business, financial condition, and results of operations.
tree_15.add_node(1, "The outcome of pending and future claims and litigation could have a material adverse impact on our business, financial condition, and results of operations.", 18, False, True, True)
# We bear the risk of cost overruns in fixed-price contracts. We may experience reduced profits or, in some cases, losses under these contracts if costs increase above our estimates. p19
tree_15.add_node(1, "We bear the risk of cost overruns in fixed-price contracts. We may experience reduced profits or, in some cases, losses under these contracts if costs increase above our estimates.", 19, False, True, True)
# Our vulnerability to the cyclical nature of the markets in which our clients operate is exacerbated during economic downturns or time of political uncertainty.
tree_15.add_node(1, "Our vulnerability to the cyclical nature of the markets in which our clients operate is exacerbated during economic downturns or time of political uncertainty.", 19, False, True, True)
# Fluctuations in commodity prices may affect our customers’ investment decisions and therefore subject us to risks of cancellation, delays in existing work, or changes in the timing and funding of new awards. p20
tree_15.add_node(1, "Fluctuations in commodity prices may affect our customers’ investment decisions and therefore subject us to risks of cancellation, delays in existing work, or changes in the timing and funding of new awards.", 20, False, True, True)
# Our continued success is dependent upon our ability to hire, retain, and utilize qualified personnel.
tree_15.add_node(1, "Our continued success is dependent upon our ability to hire, retain, and utilize qualified personnel.", 20, False, True, True)
# Contracts with the U.S. federal government and other governments and their agencies pose additional risks relating to future funding and compliance.
tree_15.add_node(1, "Contracts with the U.S. federal government and other governments and their agencies pose additional risks relating to future funding and compliance.", 20, False, True, True)
# The contracts in our backlog may be adjusted, cancelled or suspended by our clients and, therefore, our backlog is not necessarily indicative of our future revenues or earnings. Additionally, even if fully performed, our backlog is not a good indicator of our future gross margins.  p21
tree_15.add_node(1, "The contracts in our backlog may be adjusted, cancelled or suspended by our clients and, therefore, our backlog is not necessarily indicative of our future revenues or earnings. Additionally, even if fully performed, our backlog is not a good indicator of our future gross margins.", 21, False, True, True)
# Negative conditions in the credit and financial markets and delays in receiving client payments could result in liquidity problems. p22
tree_15.add_node(1, "Negative conditions in the credit and financial markets and delays in receiving client payments could result in liquidity problems.", 22, False, True, True)
# Our use of joint ventures and partnerships exposes us to risks and uncertainties, many of which are outside of our control.
tree_15.add_node(1, "Our use of joint ventures and partnerships exposes us to risks and uncertainties, many of which are outside of our control.", 22, False, True, True)
# We are dependent on third parties to complete many of our contracts. p23
tree_15.add_node(1, "We are dependent on third parties to complete many of our contracts.", 23, False, True, True)
# Maintaining adequate bonding and letter of credit capacity is necessary for us to successfully bid on and win some contracts.
tree_15.add_node(1, "Maintaining adequate bonding and letter of credit capacity is necessary for us to successfully bid on and win some contracts.", 23, False, True, True)
# Past and future environmental, heath, and safety laws could impose significant additional costs and liabilities. 
tree_15.add_node(1, "Past and future environmental, heath, and safety laws could impose significant additional costs and liabilities.", 23, False, True, True)
# If we fail to comply with federal, state, local or foreign governmental requirements, our business may be adversely affected. p24
tree_15.add_node(1, "If we fail to comply with federal, state, local or foreign governmental requirements, our business may be adversely affected.", 24, False, True, True)
# Employee, agent or partner misconduct or our overall failure to comply with laws or regulations could weaken our ability to win contracts, which could result in reduced revenues and profits.
tree_15.add_node(1, "Employee, agent or partner misconduct or our overall failure to comply with laws or regulations could weaken our ability to win contracts, which could result in reduced revenues and profits.", 24, False, True, True)
# We could be adversely affected by violations of the U.S. Foreign Corrupt Practices Act and similar worldwide anti-bribery laws.
tree_15.add_node(1, "We could be adversely affected by violations of the U.S. Foreign Corrupt Practices Act and similar worldwide anti-bribery laws.", 24, False, True, True)
# The loss of or a significant reduction in business from one or a few customers could have an adverse impact on us.
tree_15.add_node(1, "The loss of or a significant reduction in business from one or a few customers could have an adverse impact on us.", 24, False, True, True)
# We engage in a highly competitive business. If we are unable to compete effectively, we could lose market share and our business and results of operations could be negatively impacted.
tree_15.add_node(1, "We engage in a highly competitive business. If we are unable to compete effectively, we could lose market share and our business and results of operations could be negatively impacted.", 24, False, True, True)
# In addition to the risks discussed elsewhere in Risk Factors, our international operations are also exposed to additional risks and uncertainties including unfavorable political developments and weak foreign economies. p25
tree_15.add_node(1, "In addition to the risks discussed elsewhere in Risk Factors, our international operations are also exposed to additional risks and uncertainties including unfavorable political developments and weak foreign economies.", 25, False, True, True)
# We work in international locations where there are high security risks, which could result in harm to our employees or unanticipated cost.
tree_15.add_node(1, "We work in international locations where there are high security risks, which could result in harm to our employees or unanticipated cost.", 25, False, True, True)
# Foreign exchange risks may affect our ability to realize a profit from certain projects.
tree_15.add_node(1, "Foreign exchange risks may affect our ability to realize a profit from certain projects.", 25, False, True, True)
# Our business strategy relies in part on acquisitions to sustain our growth. Acquisitions of other companies present certain risks and uncertainties. p26
tree_15.add_node(1, "Our business strategy relies in part on acquisitions to sustain our growth. Acquisitions of other companies present certain risks and uncertainties.", 26, False, True, True)
# In the event we issue stock as consideration for certain acquisitions we may make, we could dilute share ownership.
tree_15.add_node(1, "In the event we issue stock as consideration for certain acquisitions we may make, we could dilute share ownership.", 26, False, True, True)
# Our quarterly results may fluctuate significantly, which could have a material negative effect on the price of our common stock.
tree_15.add_node(1, "Our quarterly results may fluctuate significantly, which could have a material negative effect on the price of our common stock.", 26, False, True, True)
# Our actual results could differ from the estimates and assumptions used to prepare our financial statements. p27
tree_15.add_node(1, "Our actual results could differ from the estimates and assumptions used to prepare our financial statements.", 27, False, True, True)
# An impairment charge of goodwill could have a material adverse impact on our financial condition and results of operations.
tree_15.add_node(1, "An impairment charge of goodwill could have a material adverse impact on our financial condition and results of operations.", 27, False, True, True)
# We may be required to contribute additional cash to meet any underfunded benefit obligations associated with retirement and post-retirement benefit plans we manage.
tree_15.add_node(1, "We may be required to contribute additional cash to meet any underfunded benefit obligations associated with retirement and post-retirement benefit plans we manage.", 27, False, True, True)
#Rising inflation, interest rates, and/or construction costs could reduce the demand for our services as well as decrease our profit on our existing contracts, in particular with respect to our fixed-price contracts. p28
tree_15.add_node(1, "Rising inflation, interest rates, and/or construction costs could reduce the demand for our services as well as decrease our profit on our existing contracts, in particular with respect to our fixed-price contracts.", 28, False, True, True)
# We may be affected by market or regulatory responses to climate change.
tree_15.add_node(1, "We may be affected by market or regulatory responses to climate change.", 28, False, True, True)
# Our effective tax rate may increase or decrease.
tree_15.add_node(1, "Our effective tax rate may increase or decrease.", 28, False, True, True)
# Systems and information technology interruption could adversely impact our ability to operate.
tree_15.add_node(1, "Systems and information technology interruption could adversely impact our ability to operate.", 28, False, True, True)
# We may not be able to protect our intellectual property or that of our clients.
tree_15.add_node(1, "We may not be able to protect our intellectual property or that of our clients.", 28, False, True, True)
# Our businesses could be materially and adversely affected by events outside of our control.p29
tree_15.add_node(1, "Our businesses could be materially and adversely affected by events outside of our control.", 29, False, True, True)
# We are subject to professional standards, duties and statutory obligations on professional reports and opinions we issue, which could subject us to monetary damages.
tree_15.add_node(1, "We are subject to professional standards, duties and statutory obligations on professional reports and opinions we issue, which could subject us to monetary damages.", 29, False, True, True)
# Delaware law and our charter documents may impede or discourage a takeover or change of control.
tree_15.add_node(1, "Delaware law and our charter documents may impede or discourage a takeover or change of control.", 29, False, True, True)

print(tree_15.show_tree_ascii())
tree_15.show_formatted_tree_in_notebook()










ITEM 1A. RISK FACTORS
├── Construction and maintenance sites are inherently dangerous workplaces. If we fail to maintain safe work sites, we can be exposed to significant financial losses as well as civil and criminal liabilities.
├── Our project execution activities may result in liability for faulty engineering services.
├── The outcome of pending and future claims and litigation could have a material adverse impact on our business, financial condition, and results of operations.
├── We bear the risk of cost overruns in fixed-price contracts. We may experience reduced profits or, in some cases, losses under these contracts if costs increase above our estimates.
├── Our vulnerability to the cyclical nature of the markets in which our clients operate is exacerbated during economic downturns or time of political uncertainty.
├── Fluctuations in commodity prices may affect our customers’ investment decisions and therefore subject us to risks of cancellation, delays in existing work, or c

- (ID 0, depth=0) **ITEM 1A. RISK FACTORS**
    - (ID 1, depth=1) ***Construction and maintenance sites are inherently dangerous workplaces. If we fail to maintain safe work sites, we can be exposed to significant financial losses as well as civil and criminal liabilities.***
    - (ID 2, depth=1) ***Our project execution activities may result in liability for faulty engineering services.***
    - (ID 3, depth=1) ***The outcome of pending and future claims and litigation could have a material adverse impact on our business, financial condition, and results of operations.***
    - (ID 4, depth=1) ***We bear the risk of cost overruns in fixed-price contracts. We may experience reduced profits or, in some cases, losses under these contracts if costs increase above our estimates.***
    - (ID 5, depth=1) ***Our vulnerability to the cyclical nature of the markets in which our clients operate is exacerbated during economic downturns or time of political uncertainty.***
    - (ID 6, depth=1) ***Fluctuations in commodity prices may affect our customers’ investment decisions and therefore subject us to risks of cancellation, delays in existing work, or changes in the timing and funding of new awards.***
    - (ID 7, depth=1) ***Our continued success is dependent upon our ability to hire, retain, and utilize qualified personnel.***
    - (ID 8, depth=1) ***Contracts with the U.S. federal government and other governments and their agencies pose additional risks relating to future funding and compliance.***
    - (ID 9, depth=1) ***The contracts in our backlog may be adjusted, cancelled or suspended by our clients and, therefore, our backlog is not necessarily indicative of our future revenues or earnings. Additionally, even if fully performed, our backlog is not a good indicator of our future gross margins.***
    - (ID 10, depth=1) ***Negative conditions in the credit and financial markets and delays in receiving client payments could result in liquidity problems.***
    - (ID 11, depth=1) ***Our use of joint ventures and partnerships exposes us to risks and uncertainties, many of which are outside of our control.***
    - (ID 12, depth=1) ***We are dependent on third parties to complete many of our contracts.***
    - (ID 13, depth=1) ***Maintaining adequate bonding and letter of credit capacity is necessary for us to successfully bid on and win some contracts.***
    - (ID 14, depth=1) ***Past and future environmental, heath, and safety laws could impose significant additional costs and liabilities.***
    - (ID 15, depth=1) ***If we fail to comply with federal, state, local or foreign governmental requirements, our business may be adversely affected.***
    - (ID 16, depth=1) ***Employee, agent or partner misconduct or our overall failure to comply with laws or regulations could weaken our ability to win contracts, which could result in reduced revenues and profits.***
    - (ID 17, depth=1) ***We could be adversely affected by violations of the U.S. Foreign Corrupt Practices Act and similar worldwide anti-bribery laws.***
    - (ID 18, depth=1) ***The loss of or a significant reduction in business from one or a few customers could have an adverse impact on us.***
    - (ID 19, depth=1) ***We engage in a highly competitive business. If we are unable to compete effectively, we could lose market share and our business and results of operations could be negatively impacted.***
    - (ID 20, depth=1) ***In addition to the risks discussed elsewhere in Risk Factors, our international operations are also exposed to additional risks and uncertainties including unfavorable political developments and weak foreign economies.***
    - (ID 21, depth=1) ***We work in international locations where there are high security risks, which could result in harm to our employees or unanticipated cost.***
    - (ID 22, depth=1) ***Foreign exchange risks may affect our ability to realize a profit from certain projects.***
    - (ID 23, depth=1) ***Our business strategy relies in part on acquisitions to sustain our growth. Acquisitions of other companies present certain risks and uncertainties.***
    - (ID 24, depth=1) ***In the event we issue stock as consideration for certain acquisitions we may make, we could dilute share ownership.***
    - (ID 25, depth=1) ***Our quarterly results may fluctuate significantly, which could have a material negative effect on the price of our common stock.***
    - (ID 26, depth=1) ***Our actual results could differ from the estimates and assumptions used to prepare our financial statements.***
    - (ID 27, depth=1) ***An impairment charge of goodwill could have a material adverse impact on our financial condition and results of operations.***
    - (ID 28, depth=1) ***We may be required to contribute additional cash to meet any underfunded benefit obligations associated with retirement and post-retirement benefit plans we manage.***
    - (ID 29, depth=1) ***Rising inflation, interest rates, and/or construction costs could reduce the demand for our services as well as decrease our profit on our existing contracts, in particular with respect to our fixed-price contracts.***
    - (ID 30, depth=1) ***We may be affected by market or regulatory responses to climate change.***
    - (ID 31, depth=1) ***Our effective tax rate may increase or decrease.***
    - (ID 32, depth=1) ***Systems and information technology interruption could adversely impact our ability to operate.***
    - (ID 33, depth=1) ***We may not be able to protect our intellectual property or that of our clients.***
    - (ID 34, depth=1) ***Our businesses could be materially and adversely affected by events outside of our control.***
    - (ID 35, depth=1) ***We are subject to professional standards, duties and statutory obligations on professional reports and opinions we issue, which could subject us to monetary damages.***
    - (ID 36, depth=1) ***Delaware law and our charter documents may impede or discourage a takeover or change of control.***

In [13]:
tree_19 = HeadingTree("ITEM 7. MANAGEMENT’S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS", 22)
# Forward-looking statements
tree_19.add_node(1, "Forward-looking statements", 22, False, True, True)
# Overview
tree_19.add_node(1, "Overview", 22, False, True, True)
# Key trends and uncertainties regarding our existing business
tree_19.add_node(1, "Key trends and uncertainties regarding our existing business", 22, False, True, True)
# CONSOLIDATED RESULTS OF OPERATIONS p24
tree_19.add_node(1, "CONSOLIDATED RESULTS OF OPERATIONS", 24, True, True, False)
# Net sales
tree_19.add_node(2, "Net sales", 24, False, True, True)
# Gross profit p25
tree_19.add_node(2, "Gross profit", 25, False, True, True)
# Selling, general and administrative (“SG&A”)
tree_19.add_node(2, "Selling, general and administrative (“SG&A”)", 25, False, True, True)
# Net interest expense p26
tree_19.add_node(2, "Net interest expense", 26, False, True, True)
# Loss on early extinguishment of debt
tree_19.add_node(2, "Loss on early extinguishment of debt", 26, False, True, True)
# Provision for income taxes
tree_19.add_node(2, "Provision for income taxes", 26, False, True, True)
# SEGMENT RESULTS OF OPERATIONS
tree_19.add_node(1, "SEGMENT RESULTS OF OPERATIONS", 26, True, True, False)
# Aquatic Systems p27
tree_19.add_node(2, "Aquatic Systems", 27, False, True, False)
# Net sales
tree_19.add_node(3, "Net sales", 27, False, True, True)
# Segment income
tree_19.add_node(3, "Segment income", 27, False, True, True)
# Filtration Solutions p28
tree_19.add_node(2, "Filtration Solutions", 28, False, True, False)
# Net sales
tree_19.add_node(3, "Net sales", 28, False, True, True)
# Segment income p29
tree_19.add_node(3, "Segment income", 29, False, True, True)
# Flow Technologies
tree_19.add_node(2, "Flow Technologies", 29, False, True, False)
# Net sales p30
tree_19.add_node(3, "Net sales", 30, False, True, True)
# Segment income
tree_19.add_node(3, "Segment income", 30, False, True, True)
# LIQUIDITY AND CAPITAL RESOURCES p31
tree_19.add_node(1, "LIQUIDITY AND CAPITAL RESOURCES", 31, True, True, False)
# Operating activities
tree_19.add_node(2, "Operating activities", 31, False, True, True)
# Investing activities
tree_19.add_node(2, "Investing activities", 31, False, True, True)
# Financing activities p32
tree_19.add_node(2, "Financing activities", 32, False, True, True)
# Authorized shares p33
tree_19.add_node(3, "Authorized shares", 33, False, False, True)
# Share repurchases
tree_19.add_node(3, "Share repurchases", 33, False, False, True)
# Dividends
tree_19.add_node(3, "Dividends", 33, False, False, True)
# Contractual obligations p34
tree_19.add_node(2, "Contractual obligations", 34, False, True, True)
# Other financial measures
tree_19.add_node(2, "Other financial measures", 34, False, True, True)
# Off-balance sheet arrangements
tree_19.add_node(2, "Off-balance sheet arrangements", 34, False, True, True)
# COMMITMENTS AND CONTINGENCIES
tree_19.add_node(1, "COMMITMENTS AND CONTINGENCIES", 34, True, True, False)
# Product liability claims
tree_19.add_node(2, "Product liability claims", 34, False, True, True)
# Stand-by letters of credit, bank guarantees and bonds
tree_19.add_node(2, "Stand-by letters of credit, bank guarantees and bonds", 34, False, True, True)
# NEW ACCOUNTING STANDARDS p35
tree_19.add_node(1, "NEW ACCOUNTING STANDARDS", 35, True, True, False)
# CRITICAL ACCOUNTING POLICIES
tree_19.add_node(1, "CRITICAL ACCOUNTING POLICIES", 35, True, True, False)
# Impairment of goodwill and indefinite-lived intangibles
tree_19.add_node(2, "Impairment of goodwill and indefinite-lived intangibles", 35, False, True, True)
# Goodwill
tree_19.add_node(3, "Goodwill", 35, False, False, True)
# Identifiable intangible assets p36
tree_19.add_node(3, "Identifiable intangible assets", 36, False, False, True)
# Pension and other post-retirement plans p37
tree_19.add_node(2, "Pension and other post-retirement plans", 37, False, True, True)
# Discount rates
tree_19.add_node(3, "Discount rates", 37, False, False, True)
# Expected rate of return
tree_19.add_node(3, "Expected rate of return", 37, False, False, True)
# Loss contingencies
tree_19.add_node(2, "Loss contingencies", 37, False, True, True)
# Income taxes
tree_19.add_node(2, "Income taxes", 37, False, True, True)

print(tree_19.show_tree_ascii())
tree_19.show_formatted_tree_in_notebook()








ITEM 7. MANAGEMENT’S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS
├── Forward-looking statements
├── Overview
├── Key trends and uncertainties regarding our existing business
├── CONSOLIDATED RESULTS OF OPERATIONS
│   ├── Net sales
│   ├── Gross profit
│   ├── Selling, general and administrative (“SG&A”)
│   ├── Net interest expense
│   ├── Loss on early extinguishment of debt
│   └── Provision for income taxes
├── SEGMENT RESULTS OF OPERATIONS
│   ├── Aquatic Systems
│   │   ├── Net sales
│   │   └── Segment income
│   ├── Filtration Solutions
│   │   ├── Net sales
│   │   └── Segment income
│   └── Flow Technologies
│       ├── Net sales
│       └── Segment income
├── LIQUIDITY AND CAPITAL RESOURCES
│   ├── Operating activities
│   ├── Investing activities
│   ├── Financing activities
│   │   ├── Authorized shares
│   │   ├── Share repurchases
│   │   └── Dividends
│   ├── Contractual obligations
│   ├── Other financial measures
│   └── Off-balance sheet a

- (ID 0, depth=0) **ITEM 7. MANAGEMENT’S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS**
    - (ID 1, depth=1) ***Forward-looking statements***
    - (ID 2, depth=1) ***Overview***
    - (ID 3, depth=1) ***Key trends and uncertainties regarding our existing business***
    - (ID 4, depth=1) **CONSOLIDATED RESULTS OF OPERATIONS**
        - (ID 5, depth=2) ***Net sales***
        - (ID 6, depth=2) ***Gross profit***
        - (ID 7, depth=2) ***Selling, general and administrative (“SG&A”)***
        - (ID 8, depth=2) ***Net interest expense***
        - (ID 9, depth=2) ***Loss on early extinguishment of debt***
        - (ID 10, depth=2) ***Provision for income taxes***
    - (ID 11, depth=1) **SEGMENT RESULTS OF OPERATIONS**
        - (ID 12, depth=2) **Aquatic Systems**
            - (ID 13, depth=3) ***Net sales***
            - (ID 14, depth=3) ***Segment income***
        - (ID 15, depth=2) **Filtration Solutions**
            - (ID 16, depth=3) ***Net sales***
            - (ID 17, depth=3) ***Segment income***
        - (ID 18, depth=2) **Flow Technologies**
            - (ID 19, depth=3) ***Net sales***
            - (ID 20, depth=3) ***Segment income***
    - (ID 21, depth=1) **LIQUIDITY AND CAPITAL RESOURCES**
        - (ID 22, depth=2) ***Operating activities***
        - (ID 23, depth=2) ***Investing activities***
        - (ID 24, depth=2) ***Financing activities***
            - (ID 25, depth=3) *Authorized shares*
            - (ID 26, depth=3) *Share repurchases*
            - (ID 27, depth=3) *Dividends*
        - (ID 28, depth=2) ***Contractual obligations***
        - (ID 29, depth=2) ***Other financial measures***
        - (ID 30, depth=2) ***Off-balance sheet arrangements***
    - (ID 31, depth=1) **COMMITMENTS AND CONTINGENCIES**
        - (ID 32, depth=2) ***Product liability claims***
        - (ID 33, depth=2) ***Stand-by letters of credit, bank guarantees and bonds***
    - (ID 34, depth=1) **NEW ACCOUNTING STANDARDS**
    - (ID 35, depth=1) **CRITICAL ACCOUNTING POLICIES**
        - (ID 36, depth=2) ***Impairment of goodwill and indefinite-lived intangibles***
            - (ID 37, depth=3) *Goodwill*
            - (ID 38, depth=3) *Identifiable intangible assets*
        - (ID 39, depth=2) ***Pension and other post-retirement plans***
            - (ID 40, depth=3) *Discount rates*
            - (ID 41, depth=3) *Expected rate of return*
        - (ID 42, depth=2) ***Loss contingencies***
        - (ID 43, depth=2) ***Income taxes***

In [14]:
tree_23 = HeadingTree("ITEM 1A. RISK FACTORS", 18)

print(tree_23.show_tree_ascii())
tree_23.show_formatted_tree_in_notebook()

ITEM 1A. RISK FACTORS


- (ID 0, depth=0) **ITEM 1A. RISK FACTORS**

In [15]:
tree_24 = HeadingTree("ITEM 1A. RISK FACTORS", 7)
# The integration of The Hillshire Brands Company may be more difficult, costly or time consuming than expected, and the acquisition may not result in any or all of the anticipated benefits, including cost synergies.
tree_24.add_node(1, "The integration of The Hillshire Brands Company may be more difficult, costly or time consuming than expected, and the acquisition may not result in any or all of the anticipated benefits, including cost synergies.", 7, False, True, False)
# Fluctuations in commodity prices and in the availability of raw materials, especially feed grains, live cattle, live swine and other inputs could negatively impact our earnings. p8
tree_24.add_node(1, "Fluctuations in commodity prices and in the availability of raw materials, especially feed grains, live cattle, live swine and other inputs could negatively impact our earnings.", 8, False, True, False)
# The prices we receive for our products may fluctuate due to competition from other food producers and processors.
tree_24.add_node(1, "The prices we receive for our products may fluctuate due to competition from other food producers and processors.", 8, False, True, False)
# Outbreaks of livestock diseases can adversely impact our ability to conduct our operations and demand for our products.
tree_24.add_node(1, "Outbreaks of livestock diseases can adversely impact our ability to conduct our operations and demand for our products.", 8, False, True, False)
# We are subject to risks associated with our international activities, which could negatively affect our sales to customers in foreign countries, as well as our operations and assets in such countries. p9
tree_24.add_node(1, "We are subject to risks associated with our international activities, which could negatively affect our sales to customers in foreign countries, as well as our operations and assets in such countries.", 9, False, True, False)
# We depend on the availability of, and good relations with, our employees.
tree_24.add_node(1, "We depend on the availability of, and good relations with, our employees.", 9, False, True, False)
# We depend on contract growers and independent producers to supply us with livestock.
tree_24.add_node(1, "We depend on contract growers and independent producers to supply us with livestock.", 9, False, True, False)
# If our products become contaminated, we may be subject to product liability claims and product recalls.
tree_24.add_node(1, "If our products become contaminated, we may be subject to product liability claims and product recalls.", 9, False, True, False)
# Changes in consumer preference and failure to maintain favorable consumer perception of our brands and products could negatively impact our business. p10
tree_24.add_node(1, "Changes in consumer preference and failure to maintain favorable consumer perception of our brands and products could negatively impact our business.", 10, False, True, False)
# Failure to continually innovate and successfully launch new products and maintain our brand image through marketing investment could adversely impact our operating results.
tree_24.add_node(1, "Failure to continually innovate and successfully launch new products and maintain our brand image through marketing investment could adversely impact our operating results.", 10, False, True, False)
# Failure to leverage our brand value propositions to compete against private label products, especially during economic downturn, may adversely affect our profitability.
tree_24.add_node(1, "Failure to leverage our brand value propositions to compete against private label products, especially during economic downturn, may adversely affect our profitability.", 10, False, True, False)
# Our level of indebtedness and the terms of our indebtedness could negatively impact our business and liquidity position. p11
tree_24.add_node(1, "Our level of indebtedness and the terms of our indebtedness could negatively impact our business and liquidity position.", 11, False, True, False)
# An impairment in the carrying value of our goodwill or indefinite life intangible assets could negatively impact our consolidated results of operations and net worth.
tree_24.add_node(1, "An impairment in the carrying value of our goodwill or indefinite life intangible assets could negatively impact our consolidated results of operations and net worth.", 11, False, True, False)
# New or more stringent domestic and international government regulations could impose material costs on us and could adversely affect our business.
tree_24.add_node(1, "New or more stringent domestic and international government regulations could impose material costs on us and could adversely affect our business.", 11, False, True, False)
# Legal claims, other regulatory enforcement actions, or failure to comply with applicable legal standards or requirements could affect our product sales, reputation and profitability. p12
tree_24.add_node(1, "Legal claims, other regulatory enforcement actions, or failure to comply with applicable legal standards or requirements could affect our product sales, reputation and profitability.", 12, False, True, False)
# The Company is subject to stringent environmental regulation and potentially subject to environmental litigation, proceedings, and investigations.
tree_24.add_node(1, "The Company is subject to stringent environmental regulation and potentially subject to environmental litigation, proceedings, and investigations.", 12, False, True, False)
# Failures or security breaches of our information technology systems could disrupt our operations and negatively impact our business.
tree_24.add_node(1, "Failures or security breaches of our information technology systems could disrupt our operations and negatively impact our business.", 12, False, True, False)
# If we pursue strategic acquisitions or divestitures, we may not be able to successfully consummate favorable transactions or successfully integrate acquired businesses.
tree_24.add_node(1, "If we pursue strategic acquisitions or divestitures, we may not be able to successfully consummate favorable transactions or successfully integrate acquired businesses.", 12, False, True, False)
# Market fluctuations could negatively impact our operating results as we hedge certain transactions. p13
tree_24.add_node(1, "Market fluctuations could negatively impact our operating results as we hedge certain transactions.", 13, False, True, False)
# Deterioration of economic conditions could negatively impact our business.
tree_24.add_node(1, "Deterioration of economic conditions could negatively impact our business.", 13, False, True, False)
# The loss of one or more of our largest customers could negatively impact our business.
tree_24.add_node(1, "The loss of one or more of our largest customers could negatively impact our business.", 13, False, True, False)
# Extreme factors or forces beyond our control could negatively impact our business.
tree_24.add_node(1, "Extreme factors or forces beyond our control could negatively impact our business.", 13, False, True, False)
# Failure to maximize or to successfully assert our intellectual property rights could impact our competitiveness.
tree_24.add_node(1, "Failure to maximize or to successfully assert our intellectual property rights could impact our competitiveness.", 13, False, True, False)
# Tyson Limited Partnership can exercise significant control. p13
tree_24.add_node(1, "Tyson Limited Partnership can exercise significant control.", 13, False, True, False)
# We may incur additional tax expense or become subject to additional tax liabilities.
tree_24.add_node(1, "We may incur additional tax expense or become subject to additional tax liabilities.", 13, False, True, False)
# We could incur substantial tax liabilities as a result of the DEMB Master Blenders 1753 N.V (“DEMB”) Spin-Off.
tree_24.add_node(1, "We could incur substantial tax liabilities as a result of the DEMB Master Blenders 1753 N.V (“DEMB”) Spin-Off.", 13, False, True, False)
# Participation in a Multiemployer Pension Plan could adversely affect our business. -15
tree_24.add_node(1, "Participation in a Multiemployer Pension Plan could adversely affect our business.", 15, False, True, False)
# Volatility in the capital markets or interest rates could adversely impact our pension costs and the funded status of our pension plans.
tree_24.add_node(1, "Volatility in the capital markets or interest rates could adversely impact our pension costs and the funded status of our pension plans.", 15, False, True, False)

print(tree_24.show_tree_ascii())
tree_24.show_formatted_tree_in_notebook()







ITEM 1A. RISK FACTORS
├── The integration of The Hillshire Brands Company may be more difficult, costly or time consuming than expected, and the acquisition may not result in any or all of the anticipated benefits, including cost synergies.
├── Fluctuations in commodity prices and in the availability of raw materials, especially feed grains, live cattle, live swine and other inputs could negatively impact our earnings.
├── The prices we receive for our products may fluctuate due to competition from other food producers and processors.
├── Outbreaks of livestock diseases can adversely impact our ability to conduct our operations and demand for our products.
├── We are subject to risks associated with our international activities, which could negatively affect our sales to customers in foreign countries, as well as our operations and assets in such countries.
├── We depend on the availability of, and good relations with, our employees.
├── We depend on contract growers and independent pr

- (ID 0, depth=0) **ITEM 1A. RISK FACTORS**
    - (ID 1, depth=1) **The integration of The Hillshire Brands Company may be more difficult, costly or time consuming than expected, and the acquisition may not result in any or all of the anticipated benefits, including cost synergies.**
    - (ID 2, depth=1) **Fluctuations in commodity prices and in the availability of raw materials, especially feed grains, live cattle, live swine and other inputs could negatively impact our earnings.**
    - (ID 3, depth=1) **The prices we receive for our products may fluctuate due to competition from other food producers and processors.**
    - (ID 4, depth=1) **Outbreaks of livestock diseases can adversely impact our ability to conduct our operations and demand for our products.**
    - (ID 5, depth=1) **We are subject to risks associated with our international activities, which could negatively affect our sales to customers in foreign countries, as well as our operations and assets in such countries.**
    - (ID 6, depth=1) **We depend on the availability of, and good relations with, our employees.**
    - (ID 7, depth=1) **We depend on contract growers and independent producers to supply us with livestock.**
    - (ID 8, depth=1) **If our products become contaminated, we may be subject to product liability claims and product recalls.**
    - (ID 9, depth=1) **Changes in consumer preference and failure to maintain favorable consumer perception of our brands and products could negatively impact our business.**
    - (ID 10, depth=1) **Failure to continually innovate and successfully launch new products and maintain our brand image through marketing investment could adversely impact our operating results.**
    - (ID 11, depth=1) **Failure to leverage our brand value propositions to compete against private label products, especially during economic downturn, may adversely affect our profitability.**
    - (ID 12, depth=1) **Our level of indebtedness and the terms of our indebtedness could negatively impact our business and liquidity position.**
    - (ID 13, depth=1) **An impairment in the carrying value of our goodwill or indefinite life intangible assets could negatively impact our consolidated results of operations and net worth.**
    - (ID 14, depth=1) **New or more stringent domestic and international government regulations could impose material costs on us and could adversely affect our business.**
    - (ID 15, depth=1) **Legal claims, other regulatory enforcement actions, or failure to comply with applicable legal standards or requirements could affect our product sales, reputation and profitability.**
    - (ID 16, depth=1) **The Company is subject to stringent environmental regulation and potentially subject to environmental litigation, proceedings, and investigations.**
    - (ID 17, depth=1) **Failures or security breaches of our information technology systems could disrupt our operations and negatively impact our business.**
    - (ID 18, depth=1) **If we pursue strategic acquisitions or divestitures, we may not be able to successfully consummate favorable transactions or successfully integrate acquired businesses.**
    - (ID 19, depth=1) **Market fluctuations could negatively impact our operating results as we hedge certain transactions.**
    - (ID 20, depth=1) **Deterioration of economic conditions could negatively impact our business.**
    - (ID 21, depth=1) **The loss of one or more of our largest customers could negatively impact our business.**
    - (ID 22, depth=1) **Extreme factors or forces beyond our control could negatively impact our business.**
    - (ID 23, depth=1) **Failure to maximize or to successfully assert our intellectual property rights could impact our competitiveness.**
    - (ID 24, depth=1) **Tyson Limited Partnership can exercise significant control.**
    - (ID 25, depth=1) **We may incur additional tax expense or become subject to additional tax liabilities.**
    - (ID 26, depth=1) **We could incur substantial tax liabilities as a result of the DEMB Master Blenders 1753 N.V (“DEMB”) Spin-Off.**
    - (ID 27, depth=1) **Participation in a Multiemployer Pension Plan could adversely affect our business.**
    - (ID 28, depth=1) **Volatility in the capital markets or interest rates could adversely impact our pension costs and the funded status of our pension plans.**

In [17]:
tree_28 = HeadingTree("ITEM 1A. RISK FACTORS", 9)
# Risks Relating to the Company Generally
tree_28.add_node(1, "Risks Relating to the Company Generally", 9, False, True, True)
# Competitive Risks
tree_28.add_node(2, "Competitive Risks", 9, False, True, True)
# An overall decline in economic activity could have a material adverse effect on the financial condition and results of operations of our businesses.
tree_28.add_node(3, "An overall decline in economic activity could have a material adverse effect on the financial condition and results of operations of our businesses.", 9, False, False, True)
# We face significant competitive pressures in each of our businesses.
tree_28.add_node(3, "We face significant competitive pressures in each of our businesses.", 9, False, False, True)
# Financial Risks p10
tree_28.add_node(2, "Financial Risks", 10, False, True, True)
# We are exposed to fluctuations in currency exchange rates that could negatively impact our financial results and cash flows.
tree_28.add_node(3, "We are exposed to fluctuations in currency exchange rates that could negatively impact our financial results and cash flows.", 10, False, False, True)
# Changes in interest rates and deterioration of credit quality could reduce the value of our cash balances and investment portfolios and adversely affect our financial condition or results.
tree_28.add_node(3, "Changes in interest rates and deterioration of credit quality could reduce the value of our cash balances and investment portfolios and adversely affect our financial condition or results.", 10, False, False, True)
# Our pension obligations could adversely affect our shareholders’ equity, net income, cash flow and liquidity.
tree_28.add_node(3, "Our pension obligations could adversely affect our shareholders’ equity, net income, cash flow and liquidity.", 10, False, False, True)
# We have debt outstanding that could adversely affect our financial flexibility. p11
tree_28.add_node(3, "We have debt outstanding that could adversely affect our financial flexibility.", 11, False, False, True)
# A decline in the credit ratings of our senior debt and commercial paper may adversely affect our borrowing costs, access to capital, and financial flexibility.
tree_28.add_node(3, "A decline in the credit ratings of our senior debt and commercial paper may adversely affect our borrowing costs, access to capital, and financial flexibility.", 11, False, False, True)
# The economic and political conditions of the countries and regions in which we operate could have an adverse impact on our business, financial condition, operating results, liquidity and prospects for growth.
tree_28.add_node(3, "The economic and political conditions of the countries and regions in which we operate could have an adverse impact on our business, financial condition, operating results, liquidity and prospects for growth.", 11, False, False, True)
# The benefits of our Redomestication may not be realized or may be offset in whole or in part by factors that we do not control. p12
tree_28.add_node(3, "The benefits of our Redomestication may not be realized or may be offset in whole or in part by factors that we do not control.", 12, False, False, True)
# Our global effective tax rate is subject to a variety of different factors, which could create volatility in that tax rate, expose us to greater than anticipated tax liabilities or cause us to adjust previously recognized tax assets and liabilities.
tree_28.add_node(3, "Our global effective tax rate is subject to a variety of different factors, which could create volatility in that tax rate, expose us to greater than anticipated tax liabilities or cause us to adjust previously recognized tax assets and liabilities.", 12, False, False, True)
# Changes in our accounting estimates and assumptions could negatively affect our financial position and results of operations. p13
tree_28.add_node(3, "Changes in our accounting estimates and assumptions could negatively affect our financial position and results of operations.", 13, False, False, True)
# We may be required to record goodwill or other long-lived asset impairment charges, which could result in a significant charge to earnings.
tree_28.add_node(3, "We may be required to record goodwill or other long-lived asset impairment charges, which could result in a significant charge to earnings.", 13, False, False, True)
# We are a holding company and, therefore, may not be able to receive dividends or other payments in needed amounts from our subsidiaries.
tree_28.add_node(3, "We are a holding company and, therefore, may not be able to receive dividends or other payments in needed amounts from our subsidiaries.", 13, False, False, True)
# Legal and Regulatory Risks
tree_28.add_node(2, "Legal and Regulatory Risks", 13, False, True, True)
# We are subject to E&O claims against us as well as other contingencies and legal proceedings, some of which, if determined unfavorably to us, could have a material adverse effect on the financial condition or results of operations of a business line or the Company as a whole.
tree_28.add_node(3, "We are subject to E&O claims against us as well as other contingencies and legal proceedings, some of which, if determined unfavorably to us, could have a material adverse effect on the financial condition or results of operations of a business line or the Company as a whole.", 13, False, False, True)
# Our businesses are subject to extensive governmental regulation, which could reduce our profitability, limit our growth, or increase competition.
tree_28.add_node(3, "Our businesses are subject to extensive governmental regulation, which could reduce our profitability, limit our growth, or increase competition.", 13, False, False, True)
# Failure to protect our intellectual property rights, or allegations that we have infringed on the intellectual property rights of others, could harm our reputation, ability to compete effectively and financial condition. p16
tree_28.add_node(3, "Failure to protect our intellectual property rights, or allegations that we have infringed on the intellectual property rights of others, could harm our reputation, ability to compete effectively and financial condition.", 16, False, False, True)
# We have less flexibility as a public limited company incorporated under the laws of England and Wales with respect to certain aspects of capital management.
tree_28.add_node(3, "We have less flexibility as a public limited company incorporated under the laws of England and Wales with respect to certain aspects of capital management.", 16, False, False, True)
# The enforcement of civil liabilities against us may be more difficult.
tree_28.add_node(3, "The enforcement of civil liabilities against us may be more difficult.", 16, False, False, True)
# We are a public limited company incorporated under the laws of England and Wales. Therefore, it may not be possible to effect service of process upon us within the United States in order to enforce judgments of U.S. courts against us based on the civil liability provisions of the U.S. federal securities laws. p17
tree_28.add_node(3, "We are a public limited company incorporated under the laws of England and Wales. Therefore, it may not be possible to effect service of process upon us within the United States in order to enforce judgments of U.S. courts against us based on the civil liability provisions of the U.S. federal securities laws.", 17, False, False, True)
# Operational and Commercial Risks
tree_28.add_node(2, "Operational and Commercial Risks", 18, False, True, True)
# Our success depends on our ability to retain and attract experienced and qualified personnel, including our senior management team and other professional personnel.
tree_28.add_node(3, "Our success depends on our ability to retain and attract experienced and qualified personnel, including our senior management team and other professional personnel.", 18, False, False, True)
# Our global operations expose us to various international risks that could adversely affect our business.
tree_28.add_node(3, "Our global operations expose us to various international risks that could adversely affect our business.", 18, False, False, True)
# The occurrence of natural or man-made disasters could result in declines in business and increases in claims that could adversely affect our financial condition and results of operations.
tree_28.add_node(3, "The occurrence of natural or man-made disasters could result in declines in business and increases in claims that could adversely affect our financial condition and results of operations.", 18, False, False, True)
# Our inability to successfully recover should we experience a disaster or other business continuity problem could cause material financial loss, loss of human capital, regulatory actions, reputational harm or legal liability. p20
tree_28.add_node(3, "Our inability to successfully recover should we experience a disaster or other business continuity problem could cause material financial loss, loss of human capital, regulatory actions, reputational harm or legal liability.", 20, False, False, True)
# We rely on complex information technology systems and networks to operate our business. Any significant system or network disruption due to a breach in the security of our information technology systems could have a negative impact on our reputation, operations, sales and operating results.
tree_28.add_node(3, "We rely on complex information technology systems and networks to operate our business. Any significant system or network disruption due to a breach in the security of our information technology systems could have a negative impact on our reputation, operations, sales and operating results.", 20, False, False, True)
# Improper disclosure of confidential, personal or proprietary data could result in regulatory scrutiny, legal liability or harm our reputation.
tree_28.add_node(3, "Improper disclosure of confidential, personal or proprietary data could result in regulatory scrutiny, legal liability or harm our reputation.", 20, False, False, True)
# Our business performance and growth plans could be negatively affected if we are not able to effectively apply technology in driving value for our clients through technology-based solutions or gain internal efficiencies through the effective application of technology and related tools. Conversely, investments in innovative product offerings may fail to yield sufficient return to cover their investments. 
tree_28.add_node(3, "Our business performance and growth plans could be negatively affected if we are not able to effectively apply technology in driving value for our clients through technology-based solutions or gain internal efficiencies through the effective application of technology and related tools. Conversely, investments in innovative product offerings may fail to yield sufficient return to cover their investments.", 20, False, False, True)
# If our clients or third parties are not satisfied with our services, we may face additional cost, loss of profit opportunities, damage to our reputation or legal liability.
tree_28.add_node(3, "If our clients or third parties are not satisfied with our services, we may face additional cost, loss of profit opportunities, damage to our reputation or legal liability.", 20, False, False, True)
# Damage to our reputation could have a material adverse effect on our business.
tree_28.add_node(3, "Damage to our reputation could have a material adverse effect on our business.", 20, False, False, True)
# We rely on third parties to perform key functions of our business operations and to provide services to our clients. These third parties may act in ways that could harm our business.
tree_28.add_node(3, "We rely on third parties to perform key functions of our business operations and to provide services to our clients. These third parties may act in ways that could harm our business.", 20, False, False, True)
# Our business is exposed to risks associated with the handling of client funds.
tree_28.add_node(3, "Our business is exposed to risks associated with the handling of client funds.", 20, False, False, True)
# In connection with the implementation of our corporate strategies, we face risks associated with the acquisition or disposition of businesses, the entry into new lines of business, the integration of acquired businesses and the growth and development of these businesses.
tree_28.add_node(3, "In connection with the implementation of our corporate strategies, we face risks associated with the acquisition or disposition of businesses, the entry into new lines of business, the integration of acquired businesses and the growth and development of these businesses.", 20, False, False, True)
# Risks Relating Primarily to Our Risk Solutions Segment
tree_28.add_node(2, "Risks Relating Primarily to Our Risk Solutions Segment", 21, False, True, True)
# Results in our Risk Solutions segment may fluctuate due to many factors, including cyclical or permanent changes in the insurance and reinsurance markets outside of our control.
tree_28.add_node(3, "Results in our Risk Solutions segment may fluctuate due to many factors, including cyclical or permanent changes in the insurance and reinsurance markets outside of our control.", 21, False, False, True)
# Our results may be adversely affected by changes in the mode of compensation in the insurance industry.
tree_28.add_node(3, "Our results may be adversely affected by changes in the mode of compensation in the insurance industry.", 21, False, False, True)
# Risks Relating Primarily to Our HR Solutions Segment
tree_28.add_node(2, "Risks Relating Primarily to Our HR Solutions Segment", 21, False, True, True)
# We are subject to various risks and uncertainties in connection with the pending sale of our Benefits Administration and HR Business Process Outsourcing (BPO) Platform.
tree_28.add_node(3, "We are subject to various risks and uncertainties in connection with the pending sale of our Benefits Administration and HR Business Process Outsourcing (BPO) Platform.", 21, False, False, True)
# The profitability of our outsourcing and consulting engagements with clients may not meet our expectations due to unexpected costs, cost overruns, early contract terminations, unrealized assumptions used in our contract bidding process or the inability to maintain our prices. p22
tree_28.add_node(3, "The profitability of our outsourcing and consulting engagements with clients may not meet our expectations due to unexpected costs, cost overruns, early contract terminations, unrealized assumptions used in our contract bidding process or the inability to maintain our prices.", 22, False, False, True)
# We might not be able to achieve the cost savings required to sustain and increase our profit margins in our HR Solutions business.
tree_28.add_node(3, "We might not be able to achieve the cost savings required to sustain and increase our profit margins in our HR Solutions business.", 22, False, False, True)
# In our investment consulting business, we advise or act on behalf of clients regarding their investments. The results of these investments are uncertain and subject to numerous factors, some of which are within our control and some which are not. Clients that experience losses or lower than expected investment returns may leave us for competitors and/or assert claims against p23
tree_28.add_node(3, "In our investment consulting business, we advise or act on behalf of clients regarding their investments. The results of these investments are uncertain and subject to numerous factors, some of which are within our control and some which are not. Clients that experience losses or lower than expected investment returns may leave us for competitors and/or assert claims against us.", 23, False, False, True)
# Risks Related to Our Ordinary Shares
tree_28.add_node(2, "Risks Related to Our Ordinary Shares", 23, False, True, True)
# Transfers of the Class A Ordinary Shares may be subject to stamp duty or SDRT in the U.K., which would increase the cost of dealing in the Class A Ordinary Shares.
tree_28.add_node(3, "Transfers of the Class A Ordinary Shares may be subject to stamp duty or SDRT in the U.K., which would increase the cost of dealing in the Class A Ordinary Shares.", 23, False, False, True)
# If the Class A Ordinary Shares are not eligible for continued deposit and clearing within the facilities of DTC, then transactions in our securities may be disrupted.
tree_28.add_node(3, "If the Class A Ordinary Shares are not eligible for continued deposit and clearing within the facilities of DTC, then transactions in our securities may be disrupted.", 23, False, False, True)

print(tree_28.show_tree_ascii())
tree_28.show_formatted_tree_in_notebook()





ITEM 1A. RISK FACTORS
└── Risks Relating to the Company Generally
    ├── Competitive Risks
    │   ├── An overall decline in economic activity could have a material adverse effect on the financial condition and results of operations of our businesses.
    │   └── We face significant competitive pressures in each of our businesses.
    ├── Financial Risks
    │   ├── We are exposed to fluctuations in currency exchange rates that could negatively impact our financial results and cash flows.
    │   ├── Changes in interest rates and deterioration of credit quality could reduce the value of our cash balances and investment portfolios and adversely affect our financial condition or results.
    │   ├── Our pension obligations could adversely affect our shareholders’ equity, net income, cash flow and liquidity.
    │   ├── We have debt outstanding that could adversely affect our financial flexibility.
    │   ├── A decline in the credit ratings of our senior debt and commercial paper may ad

- (ID 0, depth=0) **ITEM 1A. RISK FACTORS**
    - (ID 1, depth=1) ***Risks Relating to the Company Generally***
        - (ID 2, depth=2) ***Competitive Risks***
            - (ID 3, depth=3) *An overall decline in economic activity could have a material adverse effect on the financial condition and results of operations of our businesses.*
            - (ID 4, depth=3) *We face significant competitive pressures in each of our businesses.*
        - (ID 5, depth=2) ***Financial Risks***
            - (ID 6, depth=3) *We are exposed to fluctuations in currency exchange rates that could negatively impact our financial results and cash flows.*
            - (ID 7, depth=3) *Changes in interest rates and deterioration of credit quality could reduce the value of our cash balances and investment portfolios and adversely affect our financial condition or results.*
            - (ID 8, depth=3) *Our pension obligations could adversely affect our shareholders’ equity, net income, cash flow and liquidity.*
            - (ID 9, depth=3) *We have debt outstanding that could adversely affect our financial flexibility.*
            - (ID 10, depth=3) *A decline in the credit ratings of our senior debt and commercial paper may adversely affect our borrowing costs, access to capital, and financial flexibility.*
            - (ID 11, depth=3) *The economic and political conditions of the countries and regions in which we operate could have an adverse impact on our business, financial condition, operating results, liquidity and prospects for growth.*
            - (ID 12, depth=3) *The benefits of our Redomestication may not be realized or may be offset in whole or in part by factors that we do not control.*
            - (ID 13, depth=3) *Our global effective tax rate is subject to a variety of different factors, which could create volatility in that tax rate, expose us to greater than anticipated tax liabilities or cause us to adjust previously recognized tax assets and liabilities.*
            - (ID 14, depth=3) *Changes in our accounting estimates and assumptions could negatively affect our financial position and results of operations.*
            - (ID 15, depth=3) *We may be required to record goodwill or other long-lived asset impairment charges, which could result in a significant charge to earnings.*
            - (ID 16, depth=3) *We are a holding company and, therefore, may not be able to receive dividends or other payments in needed amounts from our subsidiaries.*
        - (ID 17, depth=2) ***Legal and Regulatory Risks***
            - (ID 18, depth=3) *We are subject to E&O claims against us as well as other contingencies and legal proceedings, some of which, if determined unfavorably to us, could have a material adverse effect on the financial condition or results of operations of a business line or the Company as a whole.*
            - (ID 19, depth=3) *Our businesses are subject to extensive governmental regulation, which could reduce our profitability, limit our growth, or increase competition.*
            - (ID 20, depth=3) *Failure to protect our intellectual property rights, or allegations that we have infringed on the intellectual property rights of others, could harm our reputation, ability to compete effectively and financial condition.*
            - (ID 21, depth=3) *We have less flexibility as a public limited company incorporated under the laws of England and Wales with respect to certain aspects of capital management.*
            - (ID 22, depth=3) *The enforcement of civil liabilities against us may be more difficult.*
            - (ID 23, depth=3) *We are a public limited company incorporated under the laws of England and Wales. Therefore, it may not be possible to effect service of process upon us within the United States in order to enforce judgments of U.S. courts against us based on the civil liability provisions of the U.S. federal securities laws.*
        - (ID 24, depth=2) ***Operational and Commercial Risks***
            - (ID 25, depth=3) *Our success depends on our ability to retain and attract experienced and qualified personnel, including our senior management team and other professional personnel.*
            - (ID 26, depth=3) *Our global operations expose us to various international risks that could adversely affect our business.*
            - (ID 27, depth=3) *The occurrence of natural or man-made disasters could result in declines in business and increases in claims that could adversely affect our financial condition and results of operations.*
            - (ID 28, depth=3) *Our inability to successfully recover should we experience a disaster or other business continuity problem could cause material financial loss, loss of human capital, regulatory actions, reputational harm or legal liability.*
            - (ID 29, depth=3) *We rely on complex information technology systems and networks to operate our business. Any significant system or network disruption due to a breach in the security of our information technology systems could have a negative impact on our reputation, operations, sales and operating results.*
            - (ID 30, depth=3) *Improper disclosure of confidential, personal or proprietary data could result in regulatory scrutiny, legal liability or harm our reputation.*
            - (ID 31, depth=3) *Our business performance and growth plans could be negatively affected if we are not able to effectively apply technology in driving value for our clients through technology-based solutions or gain internal efficiencies through the effective application of technology and related tools. Conversely, investments in innovative product offerings may fail to yield sufficient return to cover their investments.*
            - (ID 32, depth=3) *If our clients or third parties are not satisfied with our services, we may face additional cost, loss of profit opportunities, damage to our reputation or legal liability.*
            - (ID 33, depth=3) *Damage to our reputation could have a material adverse effect on our business.*
            - (ID 34, depth=3) *We rely on third parties to perform key functions of our business operations and to provide services to our clients. These third parties may act in ways that could harm our business.*
            - (ID 35, depth=3) *Our business is exposed to risks associated with the handling of client funds.*
            - (ID 36, depth=3) *In connection with the implementation of our corporate strategies, we face risks associated with the acquisition or disposition of businesses, the entry into new lines of business, the integration of acquired businesses and the growth and development of these businesses.*
        - (ID 37, depth=2) ***Risks Relating Primarily to Our Risk Solutions Segment***
            - (ID 38, depth=3) *Results in our Risk Solutions segment may fluctuate due to many factors, including cyclical or permanent changes in the insurance and reinsurance markets outside of our control.*
            - (ID 39, depth=3) *Our results may be adversely affected by changes in the mode of compensation in the insurance industry.*
        - (ID 40, depth=2) ***Risks Relating Primarily to Our HR Solutions Segment***
            - (ID 41, depth=3) *We are subject to various risks and uncertainties in connection with the pending sale of our Benefits Administration and HR Business Process Outsourcing (BPO) Platform.*
            - (ID 42, depth=3) *The profitability of our outsourcing and consulting engagements with clients may not meet our expectations due to unexpected costs, cost overruns, early contract terminations, unrealized assumptions used in our contract bidding process or the inability to maintain our prices.*
            - (ID 43, depth=3) *We might not be able to achieve the cost savings required to sustain and increase our profit margins in our HR Solutions business.*
            - (ID 44, depth=3) *In our investment consulting business, we advise or act on behalf of clients regarding their investments. The results of these investments are uncertain and subject to numerous factors, some of which are within our control and some which are not. Clients that experience losses or lower than expected investment returns may leave us for competitors and/or assert claims against us.*
        - (ID 45, depth=2) ***Risks Related to Our Ordinary Shares***
            - (ID 46, depth=3) *Transfers of the Class A Ordinary Shares may be subject to stamp duty or SDRT in the U.K., which would increase the cost of dealing in the Class A Ordinary Shares.*
            - (ID 47, depth=3) *If the Class A Ordinary Shares are not eligible for continued deposit and clearing within the facilities of DTC, then transactions in our securities may be disrupted.*

In [19]:
tree_29 = HeadingTree("ITEM 7. MANAGEMENT’S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS", 42)
# CRITICAL ACCOUNTING ESTIMATES AND POLICIES
tree_29.add_node(1, "CRITICAL ACCOUNTING ESTIMATES AND POLICIES", 42, True, True, False)
# Revenue Recognition
tree_29.add_node(2, "Revenue Recognition", 42, False, True, False)
# Inventories p43
tree_29.add_node(2, "Inventories", 43, False, True, False)
# Allowance for Doubtful Accounts
tree_29.add_node(2, "Allowance for Doubtful Accounts", 43, False, True, False)
# Accounting for Stock-Based Compensation Plans p44
tree_29.add_node(2, "Accounting for Stock-Based Compensation Plans", 44, False, True, False)
# Contingencies and Litigation
tree_29.add_node(2, "Contingencies and Litigation", 44, False, True, False)
# Goodwill and Intangible Assets
tree_29.add_node(2, "Goodwill and Intangible Assets", 44, False, True, False)
# Income Taxes
tree_29.add_node(2, "Income Taxes", 44, False, True, False)
# Valuation of Marketable Securities p45
tree_29.add_node(2, "Valuation of Marketable Securities", 45, False, True, False)
# Recent Accounting Pronouncements
tree_29.add_node(1, "Recent Accounting Pronouncements", 45, True, True, False)
# EXECUTIVE SUMMARY
tree_29.add_node(1, "EXECUTIVE SUMMARY", 45, True, True, False)
# Proposed Merger with Orbotech, Ltd p47
tree_29.add_node(1, "Proposed Merger with Orbotech, Ltd", 47, True, True, False)
# Revenues and Gross Margin p48
tree_29.add_node(1, "Revenues and Gross Margin", 48, True, True, False)
# Product revenues
tree_29.add_node(2, "Product revenues", 48, False, True, False)
# Service revenues
tree_29.add_node(2, "Service revenues", 48, False, True, False)
# Revenues - Top Customers
tree_29.add_node(2, "Revenues - Top Customers", 48, False, True, False)
# Revenues by region p49
tree_29.add_node(2, "Revenues by region", 49, False, True, False)
# Gross margin 
tree_29.add_node(2, "Gross margin", 49, False, True, False)
# Research and Development (“R&D”)p50
tree_29.add_node(2, "Research and Development (“R&D”)", 50, False, True, False)
# Selling, General and Administrative (“SG&A”) p51
tree_29.add_node(2, "Selling, General and Administrative (“SG&A”)", 51, False, True, False)
# Restructuring Charges
tree_29.add_node(2, "Restructuring Charges", 51, False, True, False)
# Interest Expense and Other Expense (Income), Net
tree_29.add_node(2, "Interest Expense and Other Expense (Income), Net", 51, False, True, False)
# Provision for Income Taxes p52
tree_29.add_node(2, "Provision for Income Taxes", 52, False, True, False)
# Liquidity and Capital Resources p54
tree_29.add_node(2, "Liquidity and Capital Resources", 54, False, True, False)
# Cash and Cash Equivalents and Marketable Securities:
tree_29.add_node(3, "Cash and Cash Equivalents and Marketable Securities:", 54, False, True, True)
# Cash Dividends and Special Cash Dividend:
tree_29.add_node(3, "Cash Dividends and Special Cash Dividend:", 54, False, True, True)
# Stock Repurchases:
tree_29.add_node(3, "Stock Repurchases:", 54, False, True, True)
# Fiscal Year 2018 Compared to Fiscal Year 2017
tree_29.add_node(2, "Fiscal Year 2018 Compared to Fiscal Year 2017", 54, False, True, True)
# Cash Flows from Operating Activities:
tree_29.add_node(3, "Cash Flows from Operating Activities:", 54, False, True, True)
# Cash Flows from Investing Activities:
tree_29.add_node(3, "Cash Flows from Investing Activities:", 54, False, True, True)
# Cash Flows from Financing Activities:
tree_29.add_node(3, "Cash Flows from Financing Activities:", 54, False, True, True)
# Fiscal Year 2017 Compared to Fiscal Year 2016
tree_29.add_node(2, "Fiscal Year 2017 Compared to Fiscal Year 2016", 54, False, True, True)
# Cash Flows from Operating Activities:
tree_29.add_node(3, "Cash Flows from Operating Activities:", 54, False, True, True)
# Cash Flows from Investing Activities:
tree_29.add_node(3, "Cash Flows from Investing Activities:", 54, False, True, True)
# Cash Flows from Financing Activities:
tree_29.add_node(3, "Cash Flows from Financing Activities:", 54, False, True, True)
# Senior Notes:
tree_29.add_node(3, "Senior Notes:", 54, False, True, True)
# Credit Facility (Term Loans and Unfunded Revolving Credit Facility) and Revolving Credit Facility: p55
tree_29.add_node(3, "Credit Facility (Term Loans and Unfunded Revolving Credit Facility) and Revolving Credit Facility:", 55, False, True, True)
# Contractual Obligations p 58
tree_29.add_node(2, "Contractual Obligations", 58, False, True, False)
# Working Capital:  p59
tree_29.add_node(2, "Working Capital:", 59, False, True, False)
# Off-Balance Sheet Arrangements p60
tree_29.add_node(2, "Off-Balance Sheet Arrangements", 60, False, True, False)
# Indemnification Obligations
tree_29.add_node(3, "Indemnification Obligations", 60, False, True, False)


print(tree_29.show_tree_ascii())
tree_29.show_formatted_tree_in_notebook()




ITEM 7. MANAGEMENT’S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS
├── CRITICAL ACCOUNTING ESTIMATES AND POLICIES
│   ├── Revenue Recognition
│   ├── Inventories
│   ├── Allowance for Doubtful Accounts
│   ├── Accounting for Stock-Based Compensation Plans
│   ├── Contingencies and Litigation
│   ├── Goodwill and Intangible Assets
│   ├── Income Taxes
│   └── Valuation of Marketable Securities
├── Recent Accounting Pronouncements
├── EXECUTIVE SUMMARY
├── Proposed Merger with Orbotech, Ltd
└── Revenues and Gross Margin
    ├── Product revenues
    ├── Service revenues
    ├── Revenues - Top Customers
    ├── Revenues by region
    ├── Gross margin
    ├── Research and Development (“R&D”)
    ├── Selling, General and Administrative (“SG&A”)
    ├── Restructuring Charges
    ├── Interest Expense and Other Expense (Income), Net
    ├── Provision for Income Taxes
    ├── Liquidity and Capital Resources
    │   ├── Cash and Cash Equivalents and Marketable Securitie

- (ID 0, depth=0) **ITEM 7. MANAGEMENT’S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS**
    - (ID 1, depth=1) **CRITICAL ACCOUNTING ESTIMATES AND POLICIES**
        - (ID 2, depth=2) **Revenue Recognition**
        - (ID 3, depth=2) **Inventories**
        - (ID 4, depth=2) **Allowance for Doubtful Accounts**
        - (ID 5, depth=2) **Accounting for Stock-Based Compensation Plans**
        - (ID 6, depth=2) **Contingencies and Litigation**
        - (ID 7, depth=2) **Goodwill and Intangible Assets**
        - (ID 8, depth=2) **Income Taxes**
        - (ID 9, depth=2) **Valuation of Marketable Securities**
    - (ID 10, depth=1) **RECENT ACCOUNTING PRONOUNCEMENTS**
    - (ID 11, depth=1) **EXECUTIVE SUMMARY**
    - (ID 12, depth=1) **PROPOSED MERGER WITH ORBOTECH, LTD**
    - (ID 13, depth=1) **REVENUES AND GROSS MARGIN**
        - (ID 14, depth=2) **Product revenues**
        - (ID 15, depth=2) **Service revenues**
        - (ID 16, depth=2) **Revenues - Top Customers**
        - (ID 17, depth=2) **Revenues by region**
        - (ID 18, depth=2) **Gross margin**
        - (ID 19, depth=2) **Research and Development (“R&D”)**
        - (ID 20, depth=2) **Selling, General and Administrative (“SG&A”)**
        - (ID 21, depth=2) **Restructuring Charges**
        - (ID 22, depth=2) **Interest Expense and Other Expense (Income), Net**
        - (ID 23, depth=2) **Provision for Income Taxes**
        - (ID 24, depth=2) **Liquidity and Capital Resources**
            - (ID 25, depth=3) ***Cash and Cash Equivalents and Marketable Securities:***
            - (ID 26, depth=3) ***Cash Dividends and Special Cash Dividend:***
            - (ID 27, depth=3) ***Stock Repurchases:***
        - (ID 28, depth=2) ***Fiscal Year 2018 Compared to Fiscal Year 2017***
            - (ID 29, depth=3) ***Cash Flows from Operating Activities:***
            - (ID 30, depth=3) ***Cash Flows from Investing Activities:***
            - (ID 31, depth=3) ***Cash Flows from Financing Activities:***
        - (ID 32, depth=2) ***Fiscal Year 2017 Compared to Fiscal Year 2016***
            - (ID 33, depth=3) ***Cash Flows from Operating Activities:***
            - (ID 34, depth=3) ***Cash Flows from Investing Activities:***
            - (ID 35, depth=3) ***Cash Flows from Financing Activities:***
            - (ID 36, depth=3) ***Senior Notes:***
            - (ID 37, depth=3) ***Credit Facility (Term Loans and Unfunded Revolving Credit Facility) and Revolving Credit Facility:***
        - (ID 38, depth=2) **Contractual Obligations**
        - (ID 39, depth=2) **Working Capital:**
        - (ID 40, depth=2) **Off-Balance Sheet Arrangements**
            - (ID 41, depth=3) **Indemnification Obligations**

In [20]:
tree_32 = HeadingTree("ITEM 1A. RISK FACTORS", 36)
# Regulatory, Legislative and Legal Risks
tree_32.add_node(1, "Regulatory, Legislative and Legal Risks", 36, False, True, False)
# The rates of the Companies’ principal electric transmission, distribution and generation operations and gas distribution operations are subject to regulatory review.
tree_32.add_node(2, "The rates of the Companies’ principal electric transmission, distribution and generation operations and gas distribution operations are subject to regulatory review.", 36, False, True, False)
# The Companies’ generation business may be negatively affected by possible FERC actions that could change market design in the wholesale markets or affect pricing rules or revenue calculations in the RTO markets.
tree_32.add_node(2, "The Companies’ generation business may be negatively affected by possible FERC actions that could change market design in the wholesale markets or affect pricing rules or revenue calculations in the RTO markets.", 36, False, True, False)
# The Companies are subject to complex governmental regulation, including tax regulation, that could adversely affect their results of operations and subject the Companies to monetary penalties.  p37
tree_32.add_node(2, "The Companies are subject to complex governmental regulation, including tax regulation, that could adversely affect their results of operations and subject the Companies to monetary penalties.", 37, False, True, False)
# Through the SCANA Combination, Dominion Energy acquired SCANA and DESC which have been and continue to be subject to numerous legal proceedings and ongoing governmental investigations and examinations. 
tree_32.add_node(2, "Through the SCANA Combination, Dominion Energy acquired SCANA and DESC which have been and continue to be subject to numerous legal proceedings and ongoing governmental investigations and examinations.", 37, False, True, False)
# Environmental Risks
tree_32.add_node(1, "Environmental Risks", 37, False, True, False)
# Compliance with federal and/or state requirements imposing limitations on GHG emissions or efficiency improvements, as well as Dominion Energy’s commitment to achieve net zero emissions by 2050, may result in significant compliance costs, could result in certain of the Companies’ existing electric generation units being uneconomical to maintain or operate and may depend upon technological advancements which may be beyond the Companies’ control.
tree_32.add_node(2, "Compliance with federal and/or state requirements imposing limitations on GHG emissions or efficiency improvements, as well as Dominion Energy’s commitment to achieve net zero emissions by 2050, may result in significant compliance costs, could result in certain of the Companies’ existing electric generation units being uneconomical to maintain or operate and may depend upon technological advancements which may be beyond the Companies’ control.", 37, False, True, False)
# The Companies’ operations and construction activities are subject to a number of environmental laws and regulations which impose significant compliance costs on the Companies.  p38
tree_32.add_node(2, "The Companies’ operations and construction activities are subject to a number of environmental laws and regulations which impose significant compliance costs on the Companies.", 38, False, True, False)
# The Companies are subject to risks associated with the disposal and storage of coal ash.
tree_32.add_node(2, "The Companies are subject to risks associated with the disposal and storage of coal ash.", 38, False, True, False)
# Construction Risks
tree_32.add_node(1, "Construction Risks", 38, False, True, False)
# The Companies’ infrastructure build and expansion plans often require regulatory approval, including environmental permits, before commencing construction and completing projects. The Companies may not complete the facility construction, pipeline, conversion or other infrastructure projects that they commence, or they may complete projects on materially different terms, costs or timing than initially estimated or anticipated, and they may not be able to achieve the intended benefits of any such project, if completed.
tree_32.add_node(2, "The Companies’ infrastructure build and expansion plans often require regulatory approval, including environmental permits, before commencing construction and completing projects. The Companies may not complete the facility construction, pipeline, conversion or other infrastructure projects that they commence, or they may complete projects on materially different terms, costs or timing than initially estimated or anticipated, and they may not be able to achieve the intended benefits of any such project, if completed.", 38, False, True, False)
# The development, construction and commissioning of several large-scale infrastructure projects simultaneously involves significant execution risk. p39
tree_32.add_node(2, "The development, construction and commissioning of several large-scale infrastructure projects simultaneously involves significant execution risk.", 39, False, True, False)
# Operational Risks
tree_32.add_node(1, "Operational Risks", 39, False, True, False)
# The Companies’ financial performance and condition can be affected by changes in the weather, including the effects of global climate change.
tree_32.add_node(2, "The Companies’ financial performance and condition can be affected by changes in the weather, including the effects of global climate change.", 39, False, True, False)
# The Companies’ operations are subject to operational hazards, equipment failures, supply chain disruptions and personnel issues which could negatively affect the Companies. 
tree_32.add_node(2, "The Companies’ operations are subject to operational hazards, equipment failures, supply chain disruptions and personnel issues which could negatively affect the Companies.", 39, False, True, False)
# The Companies’ financial results can be adversely affected by various factors driving supply and demand for electricity and gas and related services. p40
tree_32.add_node(2, "The Companies’ financial results can be adversely affected by various factors driving supply and demand for electricity and gas and related services.", 40, False, True, False)
# The Companies may be materially adversely affected by negative publicity.
tree_32.add_node(2, "The Companies may be materially adversely affected by negative publicity.", 40, False, True, False)
# Dominion Energy’s nonregulated generation business operates in a challenging market, which could adversely affect its results of operations and future growth. 
tree_32.add_node(2, "Dominion Energy’s nonregulated generation business operates in a challenging market, which could adversely affect its results of operations and future growth.", 40, False, True, False)
# Dominion Energy conducts certain operations through partnership arrangements involving third-party investors which may limit Dominion Energy’s operational flexibility or result in an adverse impact on its financial results. p41
tree_32.add_node(2, "Dominion Energy conducts certain operations through partnership arrangements involving third-party investors which may limit Dominion Energy’s operational flexibility or result in an adverse impact on its financial results.", 41, False, True, False)
# Hostile cyber intrusions could severely impair the Companies’ operations, lead to the disclosure of confidential information, damage the reputation of the Companies and otherwise have an adverse effect on the Companies’ business.
tree_32.add_node(2, "Hostile cyber intrusions could severely impair the Companies’ operations, lead to the disclosure of confidential information, damage the reputation of the Companies and otherwise have an adverse effect on the Companies’ business.", 41, False, True, False)
# War, acts and threats of terrorism, intentional acts and other significant events could adversely affect the Companies’ operations. p42
tree_32.add_node(2, "War, acts and threats of terrorism, intentional acts and other significant events could adversely affect the Companies’ operations.", 42, False, True, False)
# Failure to attract and retain key executive officers and an appropriately qualified workforce could have an adverse effect on the Companies’ operations.
tree_32.add_node(2, "Failure to attract and retain key executive officers and an appropriately qualified workforce could have an adverse effect on the Companies’ operations.", 43, False, True, False)
# Nuclear Generation Risks
tree_32.add_node(1, "Nuclear Generation Risks",43,  False, True, False)
# The Companies have substantial ownership interests in and operate nuclear generating units; as a result, each may incur substantial costs and liabilities.
tree_32.add_node(2, "The Companies have substantial ownership interests in and operate nuclear generating units; as a result, each may incur substantial costs and liabilities.", 43,False, True, False)
# Financial, Economic and Market Risks
tree_32.add_node(1, "Financial, Economic and Market Risks", 43, False, True, False)
# Changing rating agency requirements could negatively affect the Companies’ growth and business strategy.
tree_32.add_node(2, "Changing rating agency requirements could negatively affect the Companies’ growth and business strategy.", 43, False, True, False)
# An inability to access financial markets and, in the case of Dominion Energy, obtain cash from subsidiaries could adversely affect the execution of the Companies’ businessplans. 
tree_32.add_node(2, "An inability to access financial markets and, in the case of Dominion Energy, obtain cash from subsidiaries could adversely affect the execution of the Companies’ businessplans.", 43, False, True, False)
# Market performance, interest rates and other changes may decrease the value of the Companies’ decommissioning trust funds and Dominion Energy’s benefit plan assets or increase Dominion Energy’s liabilities, which could then require significant additional funding. 
tree_32.add_node(2, "Market performance, interest rates and other changes may decrease the value of the Companies’ decommissioning trust funds and Dominion Energy’s benefit plan assets or increase Dominion Energy’s liabilities, which could then require significant additional funding.", 43, False, True, False)
# The use of derivative instruments could result in financial losses and liquidity constraints.  p44    
tree_32.add_node(2, "The use of derivative instruments could result in financial losses and liquidity constraints.", 44, False, True, False)
# Future impairments of goodwill or other intangible assets or long-lived assets may have a material adverse effect on the Companies’ results. 
tree_32.add_node(2, "Future impairments of goodwill or other intangible assets or long-lived assets may have a material adverse effect on the Companies’ results.", 44, False, True, False)
# Exposure to counterparty performance may adversely affect the Companies’ financial results of operations. 
tree_32.add_node(2, "Exposure to counterparty performance may adversely affect the Companies’ financial results of operations.", 44, False, True, False)
# Public health crises and epidemics or pandemics, such as COVID-19, could adversely affect the Companies’ business, results of operations, financial condition, liquidity and/or cash flows.
tree_32.add_node(2, "Public health crises and epidemics or pandemics, such as COVID-19, could adversely affect the Companies’ business, results of operations, financial condition, liquidity and/or cash flows.", 44, False, True, False)


print(tree_32.show_tree_ascii())
tree_32.show_formatted_tree_in_notebook()



ITEM 1A. RISK FACTORS
├── Regulatory, Legislative and Legal Risks
│   ├── The rates of the Companies’ principal electric transmission, distribution and generation operations and gas distribution operations are subject to regulatory review.
│   ├── The Companies’ generation business may be negatively affected by possible FERC actions that could change market design in the wholesale markets or affect pricing rules or revenue calculations in the RTO markets.
│   ├── The Companies are subject to complex governmental regulation, including tax regulation, that could adversely affect their results of operations and subject the Companies to monetary penalties.
│   └── Through the SCANA Combination, Dominion Energy acquired SCANA and DESC which have been and continue to be subject to numerous legal proceedings and ongoing governmental investigations and examinations.
├── Environmental Risks
│   ├── Compliance with federal and/or state requirements imposing limitations on GHG emissions or effici

- (ID 0, depth=0) **ITEM 1A. RISK FACTORS**
    - (ID 1, depth=1) **Regulatory, Legislative and Legal Risks**
        - (ID 2, depth=2) **The rates of the Companies’ principal electric transmission, distribution and generation operations and gas distribution operations are subject to regulatory review.**
        - (ID 3, depth=2) **The Companies’ generation business may be negatively affected by possible FERC actions that could change market design in the wholesale markets or affect pricing rules or revenue calculations in the RTO markets.**
        - (ID 4, depth=2) **The Companies are subject to complex governmental regulation, including tax regulation, that could adversely affect their results of operations and subject the Companies to monetary penalties.**
        - (ID 5, depth=2) **Through the SCANA Combination, Dominion Energy acquired SCANA and DESC which have been and continue to be subject to numerous legal proceedings and ongoing governmental investigations and examinations.**
    - (ID 6, depth=1) **Environmental Risks**
        - (ID 7, depth=2) **Compliance with federal and/or state requirements imposing limitations on GHG emissions or efficiency improvements, as well as Dominion Energy’s commitment to achieve net zero emissions by 2050, may result in significant compliance costs, could result in certain of the Companies’ existing electric generation units being uneconomical to maintain or operate and may depend upon technological advancements which may be beyond the Companies’ control.**
        - (ID 8, depth=2) **The Companies’ operations and construction activities are subject to a number of environmental laws and regulations which impose significant compliance costs on the Companies.**
        - (ID 9, depth=2) **The Companies are subject to risks associated with the disposal and storage of coal ash.**
    - (ID 10, depth=1) **Construction Risks**
        - (ID 11, depth=2) **The Companies’ infrastructure build and expansion plans often require regulatory approval, including environmental permits, before commencing construction and completing projects. The Companies may not complete the facility construction, pipeline, conversion or other infrastructure projects that they commence, or they may complete projects on materially different terms, costs or timing than initially estimated or anticipated, and they may not be able to achieve the intended benefits of any such project, if completed.**
        - (ID 12, depth=2) **The development, construction and commissioning of several large-scale infrastructure projects simultaneously involves significant execution risk.**
    - (ID 13, depth=1) **Operational Risks**
        - (ID 14, depth=2) **The Companies’ financial performance and condition can be affected by changes in the weather, including the effects of global climate change.**
        - (ID 15, depth=2) **The Companies’ operations are subject to operational hazards, equipment failures, supply chain disruptions and personnel issues which could negatively affect the Companies.**
        - (ID 16, depth=2) **The Companies’ financial results can be adversely affected by various factors driving supply and demand for electricity and gas and related services.**
        - (ID 17, depth=2) **The Companies may be materially adversely affected by negative publicity.**
        - (ID 18, depth=2) **Dominion Energy’s nonregulated generation business operates in a challenging market, which could adversely affect its results of operations and future growth.**
        - (ID 19, depth=2) **Dominion Energy conducts certain operations through partnership arrangements involving third-party investors which may limit Dominion Energy’s operational flexibility or result in an adverse impact on its financial results.**
        - (ID 20, depth=2) **Hostile cyber intrusions could severely impair the Companies’ operations, lead to the disclosure of confidential information, damage the reputation of the Companies and otherwise have an adverse effect on the Companies’ business.**
        - (ID 21, depth=2) **War, acts and threats of terrorism, intentional acts and other significant events could adversely affect the Companies’ operations.**
        - (ID 22, depth=2) **Failure to attract and retain key executive officers and an appropriately qualified workforce could have an adverse effect on the Companies’ operations.**
    - (ID 23, depth=1) **Nuclear Generation Risks**
        - (ID 24, depth=2) **The Companies have substantial ownership interests in and operate nuclear generating units; as a result, each may incur substantial costs and liabilities.**
    - (ID 25, depth=1) **Financial, Economic and Market Risks**
        - (ID 26, depth=2) **Changing rating agency requirements could negatively affect the Companies’ growth and business strategy.**
        - (ID 27, depth=2) **An inability to access financial markets and, in the case of Dominion Energy, obtain cash from subsidiaries could adversely affect the execution of the Companies’ businessplans.**
        - (ID 28, depth=2) **Market performance, interest rates and other changes may decrease the value of the Companies’ decommissioning trust funds and Dominion Energy’s benefit plan assets or increase Dominion Energy’s liabilities, which could then require significant additional funding.**
        - (ID 29, depth=2) **The use of derivative instruments could result in financial losses and liquidity constraints.**
        - (ID 30, depth=2) **Future impairments of goodwill or other intangible assets or long-lived assets may have a material adverse effect on the Companies’ results.**
        - (ID 31, depth=2) **Exposure to counterparty performance may adversely affect the Companies’ financial results of operations.**
        - (ID 32, depth=2) **Public health crises and epidemics or pandemics, such as COVID-19, could adversely affect the Companies’ business, results of operations, financial condition, liquidity and/or cash flows.**

In [21]:
tree_35 = HeadingTree("ITEM 1. BUSINESS", 7)
# BUSINESS
tree_35.add_node(1, "BUSINESS", 7, True, True, False)
# Overview
tree_35.add_node(2, "Overview", 7, False, True, False)
# Business Strategy
tree_35.add_node(2, "Business Strategy", 7, False, True, False)
# Generating Reliable and Growing Cash Flows p8
tree_35.add_node(3, "Generating Reliable and Growing Cash Flows", 8, False, True, True)
# Maintaining a Balanced, Diversified Portfolio of High-Quality Assets
tree_35.add_node(3, "Maintaining a Balanced, Diversified Portfolio of High-Quality Assets", 8, False, True, True)
# Preserving Our Financial Strength, Flexibility and Liquidity
tree_35.add_node(3, "Preserving Our Financial Strength, Flexibility and Liquidity", 8, False, True, True)
# Portfolio Summary
tree_35.add_node(2, "Portfolio Summary", 8, False, True, False)
# Senior Housing Communities p9
tree_35.add_node(3, "Senior Housing Communities", 9, False, True, False)
# Medical Office Buildings
tree_35.add_node(3, "Medical Office Buildings", 9, False, True, False)
# Life Science, Research and Innovation Centers
tree_35.add_node(3, "Life Science, Research and Innovation Centers", 9, False, True, False)
# Inpatient Rehabilitation and Long-Term Acute Care Facilities
tree_35.add_node(3, "Inpatient Rehabilitation and Long-Term Acute Care Facilities", 9, False, True, False)
# Health Systems p10
tree_35.add_node(3, "Health Systems", 10, False, True, False)
# Skilled Nursing Facilities
tree_35.add_node(3, "Skilled Nursing Facilities", 10, False, True, False)
# Geographic Diversification of Properties
tree_35.add_node(3, "Geographic Diversification of Properties", 10, False, True, False)
# Loans and Investments
tree_35.add_node(3, "Loans and Investments", 10, False, True, False)
# Development and Redevelopment Projects
tree_35.add_node(3, "Development and Redevelopment Projects", 10, False, True, False)
# Segment Information
tree_35.add_node(2, "Segment Information", 10, False, True, False)
# Triple-Net Leased Properties p11
tree_35.add_node(2, "Triple-Net Leased Properties", 11, False, True, False)
# Senior Living Operations
tree_35.add_node(3, "Senior Living Operations", 11, False, True, False)
# Office Operations
tree_35.add_node(3, "Office Operations", 11, False, True, False)
# Significant Tenants and Managers
tree_35.add_node(2, "Significant Tenants and Managers", 11, False, True, False)
# Triple-Net Leased Properties
tree_35.add_node(3, "Triple-Net Leased Properties", 11, False, True, False)
# Brookdale Senior Living Leases p12
tree_35.add_node(4, "Brookdale Senior Living Leases", 12, False, False, True)
# Ardent Lease
tree_35.add_node(4, "Ardent Lease", 12, False, False, True)
# Kindred Master Leases
tree_35.add_node(4, "Kindred Master Leases", 12, False, False, True)
# Senior Living Operations p13
tree_35.add_node(3, "Senior Living Operations", 13, False, True, False)
# Competition
tree_35.add_node(2, "Competition", 13, False, True, False)
# Human Capital Management p14
tree_35.add_node(2, "Human Capital Management", 14, False, True, False)
# Talent Attraction and Retention
tree_35.add_node(3, "Talent Attraction and Retention", 14, False, True, True)
# DE&I
tree_35.add_node(3, "DE&I", 14, False, True, True)
# Health & Safety
tree_35.add_node(3, "Health & Safety", 14, False, True, True)
# Environmental, Social, and Governance
tree_35.add_node(2, "Environmental, Social, and Governance", 14, False, True, False)
# Insurance p15
tree_35.add_node(2, "Insurance", 15, False, True, False)
# Additional Information
tree_35.add_node(2, "Additional Information", 15, False, True, False)
# GOVERNMENT REGULATION
tree_35.add_node(1, "GOVERNMENT REGULATION", 15, True, True, False)
# Governmental Response to the COVID-19 Pandemic
tree_35.add_node(2, "Governmental Response to the COVID-19 Pandemic", 15, False, True, False)
# United States Healthcare Regulation, Licensing and Enforcement p16
tree_35.add_node(2, "United States Healthcare Regulation, Licensing and Enforcement", 16, False, True, False)
# Overview
tree_35.add_node(3, "Overview", 16, False, True, True)
# Licensure, Certification and CONs
tree_35.add_node(3, "Licensure, Certification and CONs", 16, False, True, True)
# Fraud and Abuse Enforcement p17
tree_35.add_node(3, "Fraud and Abuse Enforcement", 17, False, True, False)
# Reimbursement
tree_35.add_node(3, "Reimbursement", 17, False, True, False)
# Data Privacy and Security p18
tree_35.add_node(3, "Data Privacy and Security", 18, False, True, True)
# International Healthcare Regulation
tree_35.add_node(2, "International Healthcare Regulation", 18, False, True, False)
# Regulation Impacting Life Science, Research and Innovation Centers
tree_35.add_node(2, "Regulation Impacting Life Science, Research and Innovation Centers", 18, False, True, False)
# Tax Regulation p19
tree_35.add_node(2, "Tax Regulation", 19, False, True, False)
# Environmental Regulation p20
tree_35.add_node(2, "Environmental Regulation", 20, False, True, False)

print(tree_35.show_tree_ascii())
tree_35.show_formatted_tree_in_notebook()










ITEM 1. BUSINESS
├── BUSINESS
│   ├── Overview
│   ├── Business Strategy
│   │   ├── Generating Reliable and Growing Cash Flows
│   │   ├── Maintaining a Balanced, Diversified Portfolio of High-Quality Assets
│   │   └── Preserving Our Financial Strength, Flexibility and Liquidity
│   ├── Portfolio Summary
│   │   ├── Senior Housing Communities
│   │   ├── Medical Office Buildings
│   │   ├── Life Science, Research and Innovation Centers
│   │   ├── Inpatient Rehabilitation and Long-Term Acute Care Facilities
│   │   ├── Health Systems
│   │   ├── Skilled Nursing Facilities
│   │   ├── Geographic Diversification of Properties
│   │   ├── Loans and Investments
│   │   └── Development and Redevelopment Projects
│   ├── Segment Information
│   ├── Triple-Net Leased Properties
│   │   ├── Senior Living Operations
│   │   └── Office Operations
│   ├── Significant Tenants and Managers
│   │   ├── Triple-Net Leased Properties
│   │   │   ├── Brookdale Senior Living Leases
│   │   │   ├── Arde

- (ID 0, depth=0) **ITEM 1. BUSINESS**
    - (ID 1, depth=1) **BUSINESS**
        - (ID 2, depth=2) **Overview**
        - (ID 3, depth=2) **Business Strategy**
            - (ID 4, depth=3) ***Generating Reliable and Growing Cash Flows***
            - (ID 5, depth=3) ***Maintaining a Balanced, Diversified Portfolio of High-Quality Assets***
            - (ID 6, depth=3) ***Preserving Our Financial Strength, Flexibility and Liquidity***
        - (ID 7, depth=2) **Portfolio Summary**
            - (ID 8, depth=3) **Senior Housing Communities**
            - (ID 9, depth=3) **Medical Office Buildings**
            - (ID 10, depth=3) **Life Science, Research and Innovation Centers**
            - (ID 11, depth=3) **Inpatient Rehabilitation and Long-Term Acute Care Facilities**
            - (ID 12, depth=3) **Health Systems**
            - (ID 13, depth=3) **Skilled Nursing Facilities**
            - (ID 14, depth=3) **Geographic Diversification of Properties**
            - (ID 15, depth=3) **Loans and Investments**
            - (ID 16, depth=3) **Development and Redevelopment Projects**
        - (ID 17, depth=2) **Segment Information**
        - (ID 18, depth=2) **Triple-Net Leased Properties**
            - (ID 19, depth=3) **Senior Living Operations**
            - (ID 20, depth=3) **Office Operations**
        - (ID 21, depth=2) **Significant Tenants and Managers**
            - (ID 22, depth=3) **Triple-Net Leased Properties**
                - (ID 23, depth=4) *Brookdale Senior Living Leases*
                - (ID 24, depth=4) *Ardent Lease*
                - (ID 25, depth=4) *Kindred Master Leases*
            - (ID 26, depth=3) **Senior Living Operations**
        - (ID 27, depth=2) **Competition**
        - (ID 28, depth=2) **Human Capital Management**
            - (ID 29, depth=3) ***Talent Attraction and Retention***
            - (ID 30, depth=3) ***DE&I***
            - (ID 31, depth=3) ***Health & Safety***
        - (ID 32, depth=2) **Environmental, Social, and Governance**
        - (ID 33, depth=2) **Insurance**
        - (ID 34, depth=2) **Additional Information**
    - (ID 35, depth=1) **GOVERNMENT REGULATION**
        - (ID 36, depth=2) **Governmental Response to the COVID-19 Pandemic**
        - (ID 37, depth=2) **United States Healthcare Regulation, Licensing and Enforcement**
            - (ID 38, depth=3) ***Overview***
            - (ID 39, depth=3) ***Licensure, Certification and CONs***
            - (ID 40, depth=3) **Fraud and Abuse Enforcement**
            - (ID 41, depth=3) **Reimbursement**
            - (ID 42, depth=3) ***Data Privacy and Security***
        - (ID 43, depth=2) **International Healthcare Regulation**
        - (ID 44, depth=2) **Regulation Impacting Life Science, Research and Innovation Centers**
        - (ID 45, depth=2) **Tax Regulation**
        - (ID 46, depth=2) **Environmental Regulation**

In [22]:
tree_26 = HeadingTree("ITEM 2. PROPERTIES", 37)
# Generation Facilities
tree_26.add_node(1, "Generation Facilities", 37, False, True, False)
# FPL
tree_26.add_node(2, "FPL", 37, False, False, False)
# NEER p38
tree_26.add_node(2, "NEER", 38, False, False, False)
# Transmission and Distribution p41
tree_26.add_node(1, "Transmission and Distribution", 41, False, True, False)
# Character of Ownership
tree_26.add_node(1, "Character of Ownership", 41, False, True, False)

print(tree_26.show_tree_ascii())
tree_26.show_formatted_tree_in_notebook()




ITEM 2. PROPERTIES
├── Generation Facilities
│   ├── FPL
│   └── NEER
├── Transmission and Distribution
└── Character of Ownership


- (ID 0, depth=0) **ITEM 2. PROPERTIES**
    - (ID 1, depth=1) **Generation Facilities**
        - (ID 2, depth=2) FPL
        - (ID 3, depth=2) NEER
    - (ID 4, depth=1) **Transmission and Distribution**
    - (ID 5, depth=1) **Character of Ownership**

In [23]:
tree_38 = HeadingTree("ITEM 1. BUSINESS", 5)
# General Overview
tree_38.add_node(1, "General Overview", 5, True, True, False)
# Business Strategy p6
tree_38.add_node(1, "Business Strategy", 6, False, True, False)
# Internal Growth Strategies
tree_38.add_node(2, "Internal Growth Strategies", 6, False, False, True)
# Investment Strategies
tree_38.add_node(2, "Investment Strategies", 6, False, False, True)
# Financing Strategies p7
tree_38.add_node(2, "Financing Strategies", 7, False, True, True)
# Competition p8
tree_38.add_node(1, "Competition", 8, False, True, False)
# Healthcare Segments
tree_38.add_node(1, "Healthcare Segments", 8, False, True, False)
# Senior housing
tree_38.add_node(2, "Senior housing", 8, False, True, False)
# ·Independent Living Facilities
tree_38.add_node(2, "Independent Living Facilities", 8, False, False, True)
# ·Memory Care Facilities
tree_38.add_node(2, "Memory Care Facilities", 8, False, False, True)
# ·Care Homes (United Kingdom)
tree_38.add_node(2, "Care Homes (United Kingdom)", 8, False, False, True)
# ·Continuing Care Retirement Communities p9
tree_38.add_node(2, "Continuing Care Retirement Communities", 9, False, True, True)
# Post-acute/skilled nursing
tree_38.add_node(2, "Post-acute/skilled nursing", 9, False, False, True)
# Life science p10
tree_38.add_node(2, "Life science", 10, False, True, False)
# Medical office
tree_38.add_node(2, "Medical office", 10, False, True, False)
# Hospital p11
tree_38.add_node(2, "Hospital", 11, False, True, False)
# Sustainability
tree_38.add_node(1, "Sustainability", 11, False, True, False)
# Insurance
tree_38.add_node(1, "Insurance", 11, False, True, False)
# Employees of HCP p12
tree_38.add_node(1, "Employees of HCP", 12, False, True, False)
# Government Regulation, Licensing and Enforcement
tree_38.add_node(1, "Government Regulation, Licensing and Enforcement", 12, False, True, False)
# Overview
tree_38.add_node(2, "Overview", 12, False, True, False)
# Fraud and Abuse Enforcement
tree_38.add_node(2, "Fraud and Abuse Enforcement", 12, False, True, False)
# Healthcare Licensure and Certificate of Need p13
tree_38.add_node(2, "Healthcare Licensure and Certificate of Need", 13, False, True, True)
# Life Science Facilities
tree_38.add_node(2, "Life Science Facilities", 13, False, True, False)
# Senior Housing Entrance Fee Communities
tree_38.add_node(2, "Senior Housing Entrance Fee Communities", 13, False, True, False)
# Americans with Disabilities Act (the “ADA”)
tree_38.add_node(2, "Americans with Disabilities Act (the “ADA”)", 13, False, True, False)
# Environmental Matters
tree_38.add_node(2, "Environmental Matters", 13, False, False, True)
# Available Information p14
tree_38.add_node(1, "Available Information", 14, False, True, False)

print(tree_38.show_tree_ascii())
tree_38.show_formatted_tree_in_notebook()










ITEM 1. BUSINESS
├── General Overview
├── Business Strategy
│   ├── Internal Growth Strategies
│   ├── Investment Strategies
│   └── Financing Strategies
├── Competition
├── Healthcare Segments
│   ├── Senior housing
│   ├── Independent Living Facilities
│   ├── Memory Care Facilities
│   ├── Care Homes (United Kingdom)
│   ├── Continuing Care Retirement Communities
│   ├── Post-acute/skilled nursing
│   ├── Life science
│   ├── Medical office
│   └── Hospital
├── Sustainability
├── Insurance
├── Employees of HCP
├── Government Regulation, Licensing and Enforcement
│   ├── Overview
│   ├── Fraud and Abuse Enforcement
│   ├── Healthcare Licensure and Certificate of Need
│   ├── Life Science Facilities
│   ├── Senior Housing Entrance Fee Communities
│   ├── Americans with Disabilities Act (the “ADA”)
│   └── Environmental Matters
└── Available Information


- (ID 0, depth=0) **ITEM 1. BUSINESS**
    - (ID 1, depth=1) **GENERAL OVERVIEW**
    - (ID 2, depth=1) **Business Strategy**
        - (ID 3, depth=2) *Internal Growth Strategies*
        - (ID 4, depth=2) *Investment Strategies*
        - (ID 5, depth=2) ***Financing Strategies***
    - (ID 6, depth=1) **Competition**
    - (ID 7, depth=1) **Healthcare Segments**
        - (ID 8, depth=2) **Senior housing**
        - (ID 9, depth=2) *Independent Living Facilities*
        - (ID 10, depth=2) *Memory Care Facilities*
        - (ID 11, depth=2) *Care Homes (United Kingdom)*
        - (ID 12, depth=2) ***Continuing Care Retirement Communities***
        - (ID 13, depth=2) *Post-acute/skilled nursing*
        - (ID 14, depth=2) **Life science**
        - (ID 15, depth=2) **Medical office**
        - (ID 16, depth=2) **Hospital**
    - (ID 17, depth=1) **Sustainability**
    - (ID 18, depth=1) **Insurance**
    - (ID 19, depth=1) **Employees of HCP**
    - (ID 20, depth=1) **Government Regulation, Licensing and Enforcement**
        - (ID 21, depth=2) **Overview**
        - (ID 22, depth=2) **Fraud and Abuse Enforcement**
        - (ID 23, depth=2) ***Healthcare Licensure and Certificate of Need***
        - (ID 24, depth=2) **Life Science Facilities**
        - (ID 25, depth=2) **Senior Housing Entrance Fee Communities**
        - (ID 26, depth=2) **Americans with Disabilities Act (the “ADA”)**
        - (ID 27, depth=2) *Environmental Matters*
    - (ID 28, depth=1) **Available Information**

In [24]:
tree_41 = HeadingTree("ITEM 7. MANAGEMENT’S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS", 20)
# OVERVIEW
tree_41.add_node(1, "OVERVIEW", 20, True, True, False)
# RESULTS OF OPERATIONS
tree_41.add_node(1, "RESULTS OF OPERATIONS", 21, True, True, False)
# FISCAL 2018 COMPARED TO FISCAL 2017 p43
tree_41.add_node(2, "FISCAL 2018 COMPARED TO FISCAL 2017", 21, True, True, False)
# REVENUE
tree_41.add_node(2, "REVENUE", 21, True, True, False)
# Services and Support Revenue 
tree_41.add_node(3, "Services and Support Revenue", 21, False, True, False)
# Processing Revenue p22
tree_41.add_node(3, "Processing Revenue", 22, False, True, False)
# OPERATING EXPENSES
tree_41.add_node(2, "OPERATING EXPENSES", 22, True, True, False)
# Cost of Revenue
tree_41.add_node(3, "Cost of Revenue", 22, False, True, False)
# Research & Development
tree_41.add_node(3, "Research & Development", 22, False, True, False)
# Selling, General, and Administrative
tree_41.add_node(3, "Selling, General, and Administrative", 22, False, True, False)
# Gains on Disposal of Businesses p23
tree_41.add_node(3, "Gains on Disposal of Businesses", 23, False, True, False)
# INTEREST INCOME AND EXPENSE
tree_41.add_node(2, "INTEREST INCOME AND EXPENSE", 23, True, True, False)
# PROVISION FOR INCOME TAXES
tree_41.add_node(2, "PROVISION FOR INCOME TAXES", 23, True, True, False)
# NET INCOME
tree_41.add_node(2, "NET INCOME", 23, True, True, False)
# FISCAL 2017 COMPARED TO FISCAL 2016
tree_41.add_node(2, "FISCAL 2017 COMPARED TO FISCAL 2016", 23, True, True, False)
# REVENUE
tree_41.add_node(2, "REVENUE", 23, True, True, False)
# Services and Support 
tree_41.add_node(3, "Services and Support", 23, False, True, False)
# Processing p24
tree_41.add_node(3, "Processing", 24, False, True, False)
# OPERATING EXPENSES
tree_41.add_node(2, "OPERATING EXPENSES", 24, True, True, False)
# Cost of Revenue
tree_41.add_node(3, "Cost of Revenue", 24, False, True, False)
# Research & Development
tree_41.add_node(3, "Research & Development", 24, False, True, False)
# Selling, General, and Administrative 
tree_41.add_node(3, "Selling, General, and Administrative", 24, False, True, False)
# Gains on Disposal of Businesses 
tree_41.add_node(3, "Gains on Disposal of Businesses", 24, False, True, False)
# INTEREST INCOME AND EXPENSE p25
tree_41.add_node(2, "INTEREST INCOME AND EXPENSE", 25, True, True, False)
# PROVISION FOR INCOME TAXES
tree_41.add_node(2, "PROVISION FOR INCOME TAXES", 25, True, True, False)
# NET INCOME
tree_41.add_node(2, "NET INCOME", 25, True, True, False)
# REPORTABLE SEGMENT DISCUSSION
tree_41.add_node(1, "REPORTABLE SEGMENT DISCUSSION", 25, True, True, False)
# Core
tree_41.add_node(2, "Core", 25, False, True, False)
# Payments p26
tree_41.add_node(2, "Payments", 26, False, True, False)
# Complementary
tree_41.add_node(2, "Complementary", 26, False, True, False)
# Corporate and Other
tree_41.add_node(2, "Corporate and Other", 26, False, True, False)
# LIQUIDITY AND CAPITAL RESOURCES
tree_41.add_node(1, "LIQUIDITY AND CAPITAL RESOURCES", 26, True, True, False)
# Capital Requirements and Resources p27
tree_41.add_node(2, "Capital Requirements and Resources", 27, False, True, False)
# OFF BALANCE SHEET ARRANGEMENTS AND CONTRACTUAL OBLIGATIONS p28
tree_41.add_node(1, "OFF BALANCE SHEET ARRANGEMENTS AND CONTRACTUAL OBLIGATIONS", 28, True, True, False)
# RECENT ACCOUNTING PRONOUNCEMENTS
tree_41.add_node(1, "RECENT ACCOUNTING PRONOUNCEMENTS", 28, True, True, False)
# CRITICAL ACCOUNTING POLICIES p30
tree_41.add_node(1, "CRITICAL ACCOUNTING POLICIES", 30, True, True, False)
# Revenue Recognition 
tree_41.add_node(2, "Revenue Recognition", 30, False, True, False)
# Deferred Costs p31
tree_41.add_node(2, "Deferred Costs", 31, False, True, False)
# Depreciation and Amortization Expense
tree_41.add_node(2, "Depreciation and Amortization Expense", 31, False, True, False)
# Capitalization of software development costs
tree_41.add_node(2, "Capitalization of software development costs", 31, False, True, False)
# Estimates used to determine current and deferred income taxes p32
tree_41.add_node(2, "Estimates used to determine current and deferred income taxes", 32, False, True, False)
# Assumptions related to purchase accounting and goodwill
tree_41.add_node(2, "Assumptions related to purchase accounting and goodwill", 32, False, True, False)

print(tree_41.show_tree_ascii())
tree_41.show_formatted_tree_in_notebook()






ITEM 7. MANAGEMENT’S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS
├── OVERVIEW
├── RESULTS OF OPERATIONS
│   ├── FISCAL 2018 COMPARED TO FISCAL 2017
│   ├── REVENUE
│   │   ├── Services and Support Revenue
│   │   └── Processing Revenue
│   ├── OPERATING EXPENSES
│   │   ├── Cost of Revenue
│   │   ├── Research & Development
│   │   ├── Selling, General, and Administrative
│   │   └── Gains on Disposal of Businesses
│   ├── INTEREST INCOME AND EXPENSE
│   ├── PROVISION FOR INCOME TAXES
│   ├── NET INCOME
│   ├── FISCAL 2017 COMPARED TO FISCAL 2016
│   ├── REVENUE
│   │   ├── Services and Support
│   │   └── Processing
│   ├── OPERATING EXPENSES
│   │   ├── Cost of Revenue
│   │   ├── Research & Development
│   │   ├── Selling, General, and Administrative
│   │   └── Gains on Disposal of Businesses
│   ├── INTEREST INCOME AND EXPENSE
│   ├── PROVISION FOR INCOME TAXES
│   └── NET INCOME
├── REPORTABLE SEGMENT DISCUSSION
│   ├── Core
│   ├── Payments
│   ├── C

- (ID 0, depth=0) **ITEM 7. MANAGEMENT’S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS**
    - (ID 1, depth=1) **OVERVIEW**
    - (ID 2, depth=1) **RESULTS OF OPERATIONS**
        - (ID 3, depth=2) **FISCAL 2018 COMPARED TO FISCAL 2017**
        - (ID 4, depth=2) **REVENUE**
            - (ID 5, depth=3) **Services and Support Revenue**
            - (ID 6, depth=3) **Processing Revenue**
        - (ID 7, depth=2) **OPERATING EXPENSES**
            - (ID 8, depth=3) **Cost of Revenue**
            - (ID 9, depth=3) **Research & Development**
            - (ID 10, depth=3) **Selling, General, and Administrative**
            - (ID 11, depth=3) **Gains on Disposal of Businesses**
        - (ID 12, depth=2) **INTEREST INCOME AND EXPENSE**
        - (ID 13, depth=2) **PROVISION FOR INCOME TAXES**
        - (ID 14, depth=2) **NET INCOME**
        - (ID 15, depth=2) **FISCAL 2017 COMPARED TO FISCAL 2016**
        - (ID 16, depth=2) **REVENUE**
            - (ID 17, depth=3) **Services and Support**
            - (ID 18, depth=3) **Processing**
        - (ID 19, depth=2) **OPERATING EXPENSES**
            - (ID 20, depth=3) **Cost of Revenue**
            - (ID 21, depth=3) **Research & Development**
            - (ID 22, depth=3) **Selling, General, and Administrative**
            - (ID 23, depth=3) **Gains on Disposal of Businesses**
        - (ID 24, depth=2) **INTEREST INCOME AND EXPENSE**
        - (ID 25, depth=2) **PROVISION FOR INCOME TAXES**
        - (ID 26, depth=2) **NET INCOME**
    - (ID 27, depth=1) **REPORTABLE SEGMENT DISCUSSION**
        - (ID 28, depth=2) **Core**
        - (ID 29, depth=2) **Payments**
        - (ID 30, depth=2) **Complementary**
        - (ID 31, depth=2) **Corporate and Other**
    - (ID 32, depth=1) **LIQUIDITY AND CAPITAL RESOURCES**
        - (ID 33, depth=2) **Capital Requirements and Resources**
    - (ID 34, depth=1) **OFF BALANCE SHEET ARRANGEMENTS AND CONTRACTUAL OBLIGATIONS**
    - (ID 35, depth=1) **RECENT ACCOUNTING PRONOUNCEMENTS**
    - (ID 36, depth=1) **CRITICAL ACCOUNTING POLICIES**
        - (ID 37, depth=2) **Revenue Recognition**
        - (ID 38, depth=2) **Deferred Costs**
        - (ID 39, depth=2) **Depreciation and Amortization Expense**
        - (ID 40, depth=2) **Capitalization of software development costs**
        - (ID 41, depth=2) **Estimates used to determine current and deferred income taxes**
        - (ID 42, depth=2) **Assumptions related to purchase accounting and goodwill**

In [25]:
tree_46 = HeadingTree("ITEM 7. MANAGEMENT’S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS", 21)
# Critical Accounting Policies
tree_46.add_node(1, "Critical Accounting Policies", 21, False, False, False)
# Revenue Recognition and Accounts Receivable
tree_46.add_node(2, "Revenue Recognition and Accounts Receivable", 21, False, False, False)
# Real Estate
tree_46.add_node(2, "Real Estate", 21, False, False, False)
# Investments in Unconsolidated Joint Ventures p22
tree_46.add_node(2, "Investments in Unconsolidated Joint Ventures", 22, False, False, False)
# Realizability of Deferred Tax Assets and Uncertain Tax Positions
tree_46.add_node(2, "Realizability of Deferred Tax Assets and Uncertain Tax Positions", 22, False, False, False)
# Executive Overview p23
tree_46.add_node(1, "Executive Overview", 23, False, False, False)
# Financial and Portfolio Information:
tree_46.add_node(2, "Financial and Portfolio Information:", 23, False, False, False)
# Acquisition Activity (see Footnotes 3 and 7 of the Notes to Consolidated Financial Statements included in this Form 10-K):
tree_46.add_node(2, "Acquisition Activity (see Footnotes 3 and 7 of the Notes to Consolidated Financial Statements included in this Form 10-K):", 23, False, False, False)
# Disposition Activity (see Footnote 5 of the Notes to Consolidated Financial Statements included in this Form 10-K):
tree_46.add_node(2, "Disposition Activity (see Footnote 5 of the Notes to Consolidated Financial Statements included in this Form 10-K):", 23, False, False, False)
# Capital Activity (for additional details see Liquidity and Capital Resources below):
tree_46.add_node(2, "Capital Activity (for additional details see Liquidity and Capital Resources below):", 23, False, False, False)
# Results of Operations
tree_46.add_node(1, "Results of Operations", 23, False, False, False)
# Comparison of Years Ended December 31, 2017 to 2016
tree_46.add_node(2, "Comparison of Years Ended December 31, 2017 to 2016", 23, False, True, False)
# Revenue from rental properties
tree_46.add_node(3, "Revenue from rental properties", 23, False, False, False)
# Real estate taxes
tree_46.add_node(3, "Real estate taxes", 23, False, False, True)
# Impairment charges p26
tree_46.add_node(3, "Impairment charges", 26, False, False, True)
# Depreciation and amortization
tree_46.add_node(3, "Depreciation and amortization", 26, False, False, True)
# Other (expense)/income, net
tree_46.add_node(3, "Other (expense)/income, net", 26, False, False, True)
# Early extinguishment of debt charges
tree_46.add_node(3, "Early extinguishment of debt charges", 26, False, False, True)
# Benefit/(provision) for income taxes, net
tree_46.add_node(3, "Benefit/(provision) for income taxes, net", 26, False, False, True)
# Equity in income of joint ventures, net
tree_46.add_node(3, "Equity in income of joint ventures, net", 26, False, False, True)
# Gain on change in control of interests
tree_46.add_node(3, "Gain on change in control of interests", 26, False, False, True)
# Equity in income from other real estate investments, net
tree_46.add_node(3, "Equity in income from other real estate investments, net", 26, False, False, True)
# Gain on sale of operating properties, net of tax 
tree_46.add_node(3, "Gain on sale of operating properties, net of tax", 26, False, False, True)
# Net income attributable to noncontrolling interests p27
tree_46.add_node(3, "Net income attributable to noncontrolling interests", 27, False, False, True)
# Preferred stock redemption charges
tree_46.add_node(3, "Preferred stock redemption charges", 27, False, False, True)
# Net income available to the company’s common shareholders and Diluted earnings per share
tree_46.add_node(3, "Net income available to the company’s common shareholders and Diluted earnings per share", 27, False, False, True)
# Comparison of Years Ended December 31, 2016 to 2015
tree_46.add_node(2, "Comparison of Years Ended December 31, 2016 to 2015", 27, False, True, False)
# Revenue from rental properties =28
tree_46.add_node(3, "Revenue from rental properties", 28, False, False, False)
# Management and other fee income
tree_46.add_node(3, "Management and other fee income", 28, False, False, True)
# Operating and maintenance expense
tree_46.add_node(3, "Operating and maintenance expense", 28, False, False, True)
# General and administrative expenses
tree_46.add_node(3, "General and administrative expenses", 28, False, False, True)
# Impairment charges
tree_46.add_node(3, "Impairment charges", 28, False, False, True)
# Depreciation and amortization
tree_46.add_node(3, "Depreciation and amortization", 28, False, False, True)
# Interest, dividends and other investment income
tree_46.add_node(3, "Interest, dividends and other investment income", 28, False, False, True)
# Interest expense
tree_46.add_node(3, "Interest expense", 28, False, False, True)
# Early extinguishment of debt charges
tree_46.add_node(3, "Early extinguishment of debt charges", 28, False, False, True)
# Provision for income taxes, net
tree_46.add_node(3, "Provision for income taxes, net", 28, False, False, True)
# Equity in income of joint ventures, net p29
tree_46.add_node(3, "Equity in income of joint ventures, net", 29, False, False, True)
# Gain on change in control of interests
tree_46.add_node(3, "Gain on change in control of interests", 29, False, False, True)
# Equity in income from other real estate investments, net
tree_46.add_node(3, "Equity in income from other real estate investments, net", 29, False, False, True)
# Gain on sale of operating properties, net of tax
tree_46.add_node(3, "Gain on sale of operating properties, net of tax", 29, False, False, True)
# Net income available to the Company’s common shareholders and Diluted earnings per share
tree_46.add_node(3, "Net income available to the Company’s common shareholders and Diluted earnings per share", 29, False, False, True)
# Liquidity and Capital Resources
tree_46.add_node(1, "Liquidity and Capital Resources", 29, False, False, False)
# Operating Activities p30
tree_46.add_node(1, "Operating Activities", 30, False, False, False)
# Investing Activities
tree_46.add_node(1, "Investing Activities", 30, False, False, False)
# Cash outflows
tree_46.add_node(2, "Cash outflows", 30, False, False, True)
# Cash inflows
tree_46.add_node(2, "Cash inflows", 30, False, False, True)
# Cash outflows:
tree_46.add_node(2, "Cash outflows:", 30, False, False, True)
# Cash inflows
tree_46.add_node(2, "Cash inflows", 30, False, False, True)
# Acquisitions of Operating Real Estate and Other Related Net Assets p31
tree_46.add_node(2, "Acquisitions of Operating Real Estate and Other Related Net Assets", 31, False, False, False)
# Improvements to Operating Real Estate
tree_46.add_node(2, "Improvements to Operating Real Estate", 31, False, False, False)
# Improvements to Real Estate Under Development
tree_46.add_node(2, "Improvements to Real Estate Under Development", 31, False, False, False)
# Financing Activities
tree_46.add_node(1, "Financing Activities", 31, False, False, False)
# Cash outflows
tree_46.add_node(2, "Cash outflows", 30, False, False, True)
# Cash inflows
tree_46.add_node(2, "Cash inflows", 30, False, False, True)
# Cash outflows:
tree_46.add_node(2, "Cash outflows:", 30, False, False, True)
# Cash inflows
tree_46.add_node(2, "Cash inflows", 30, False, False, True)
# Preferred Stock- [ 33
tree_46.add_node(2, "Preferred Stock", 33, False, False, False)
# Common Stock
tree_46.add_node(2, "Common Stock", 33, False, False, False)
# Medium Term Notes (“MTN”) and Senior Notes
tree_46.add_node(2, "Medium Term Notes (“MTN”) and Senior Notes", 33, False, False, False)
# Credit Facility
tree_46.add_node(2, "Credit Facility", 33, False, False, False)
# Term Loan
tree_46.add_node(2, "Term Loan", 33, False, False, False)
# Mortgages Payable - 34
tree_46.add_node(2, "Mortgages Payable", 34, False, False, False)
# Dividends
tree_46.add_node(2, "Dividends", 34, False, False, False)
# Hurricane Impact
tree_46.add_node(2, "Hurricane Impact", 34, False, False, False)
# Income Taxes
tree_46.add_node(2, "Income Taxes", 34, False, False, False)
# Contractual Obligations and Other Commitments p36
tree_46.add_node(1, "Contractual Obligations and Other Commitments", 36, False, False, False)
# Off-Balance Sheet Arrangements p37
tree_46.add_node(1, "Off-Balance Sheet Arrangements", 37, False, False, False)
# Unconsolidated Real Estate Joint Ventures
tree_46.add_node(2, "Unconsolidated Real Estate Joint Ventures", 37, False, False, False)
# Other Real Estate Investments
tree_46.add_node(2, "Other Real Estate Investments", 37, False, False, False)
# Funds From Operations
tree_46.add_node(1, "Funds From Operations", 37, False, False, False)
# Same Property Net Operating Income (“Same property NOI”)p48
tree_46.add_node(1, "Same Property Net Operating Income (“Same property NOI”)", 48, False, False, False)
# Effects of Inflation
tree_46.add_node(1, "Effects of Inflation", 38, False, False, False)
# New Accounting Pronouncements
tree_46.add_node(1, "New Accounting Pronouncements", 38, False, False, False)

print(tree_46.show_tree_ascii())
tree_46.show_formatted_tree_in_notebook()












ITEM 7. MANAGEMENT’S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS
├── Critical Accounting Policies
│   ├── Revenue Recognition and Accounts Receivable
│   ├── Real Estate
│   ├── Investments in Unconsolidated Joint Ventures
│   └── Realizability of Deferred Tax Assets and Uncertain Tax Positions
├── Executive Overview
│   ├── Financial and Portfolio Information:
│   ├── Acquisition Activity (see Footnotes 3 and 7 of the Notes to Consolidated Financial Statements included in this Form 10-K):
│   ├── Disposition Activity (see Footnote 5 of the Notes to Consolidated Financial Statements included in this Form 10-K):
│   └── Capital Activity (for additional details see Liquidity and Capital Resources below):
├── Results of Operations
│   ├── Comparison of Years Ended December 31, 2017 to 2016
│   │   ├── Revenue from rental properties
│   │   ├── Real estate taxes
│   │   ├── Impairment charges
│   │   ├── Depreciation and amortization
│   │   ├── Other (expense)

- (ID 0, depth=0) **ITEM 7. MANAGEMENT’S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS**
    - (ID 1, depth=1) Critical Accounting Policies
        - (ID 2, depth=2) Revenue Recognition and Accounts Receivable
        - (ID 3, depth=2) Real Estate
        - (ID 4, depth=2) Investments in Unconsolidated Joint Ventures
        - (ID 5, depth=2) Realizability of Deferred Tax Assets and Uncertain Tax Positions
    - (ID 6, depth=1) Executive Overview
        - (ID 7, depth=2) Financial and Portfolio Information:
        - (ID 8, depth=2) Acquisition Activity (see Footnotes 3 and 7 of the Notes to Consolidated Financial Statements included in this Form 10-K):
        - (ID 9, depth=2) Disposition Activity (see Footnote 5 of the Notes to Consolidated Financial Statements included in this Form 10-K):
        - (ID 10, depth=2) Capital Activity (for additional details see Liquidity and Capital Resources below):
    - (ID 11, depth=1) Results of Operations
        - (ID 12, depth=2) **Comparison of Years Ended December 31, 2017 to 2016**
            - (ID 13, depth=3) Revenue from rental properties
            - (ID 14, depth=3) *Real estate taxes*
            - (ID 15, depth=3) *Impairment charges*
            - (ID 16, depth=3) *Depreciation and amortization*
            - (ID 17, depth=3) *Other (expense)/income, net*
            - (ID 18, depth=3) *Early extinguishment of debt charges*
            - (ID 19, depth=3) *Benefit/(provision) for income taxes, net*
            - (ID 20, depth=3) *Equity in income of joint ventures, net*
            - (ID 21, depth=3) *Gain on change in control of interests*
            - (ID 22, depth=3) *Equity in income from other real estate investments, net*
            - (ID 23, depth=3) *Gain on sale of operating properties, net of tax*
            - (ID 24, depth=3) *Net income attributable to noncontrolling interests*
            - (ID 25, depth=3) *Preferred stock redemption charges*
            - (ID 26, depth=3) *Net income available to the company’s common shareholders and Diluted earnings per share*
        - (ID 27, depth=2) **Comparison of Years Ended December 31, 2016 to 2015**
            - (ID 28, depth=3) Revenue from rental properties
            - (ID 29, depth=3) *Management and other fee income*
            - (ID 30, depth=3) *Operating and maintenance expense*
            - (ID 31, depth=3) *General and administrative expenses*
            - (ID 32, depth=3) *Impairment charges*
            - (ID 33, depth=3) *Depreciation and amortization*
            - (ID 34, depth=3) *Interest, dividends and other investment income*
            - (ID 35, depth=3) *Interest expense*
            - (ID 36, depth=3) *Early extinguishment of debt charges*
            - (ID 37, depth=3) *Provision for income taxes, net*
            - (ID 38, depth=3) *Equity in income of joint ventures, net*
            - (ID 39, depth=3) *Gain on change in control of interests*
            - (ID 40, depth=3) *Equity in income from other real estate investments, net*
            - (ID 41, depth=3) *Gain on sale of operating properties, net of tax*
            - (ID 42, depth=3) *Net income available to the Company’s common shareholders and Diluted earnings per share*
    - (ID 43, depth=1) Liquidity and Capital Resources
    - (ID 44, depth=1) Operating Activities
    - (ID 45, depth=1) Investing Activities
        - (ID 46, depth=2) *Cash outflows*
        - (ID 47, depth=2) *Cash inflows*
        - (ID 48, depth=2) *Cash outflows:*
        - (ID 49, depth=2) *Cash inflows*
        - (ID 50, depth=2) Acquisitions of Operating Real Estate and Other Related Net Assets
        - (ID 51, depth=2) Improvements to Operating Real Estate
        - (ID 52, depth=2) Improvements to Real Estate Under Development
    - (ID 53, depth=1) Financing Activities
        - (ID 54, depth=2) *Cash outflows*
        - (ID 55, depth=2) *Cash inflows*
        - (ID 56, depth=2) *Cash outflows:*
        - (ID 57, depth=2) *Cash inflows*
        - (ID 58, depth=2) Preferred Stock
        - (ID 59, depth=2) Common Stock
        - (ID 60, depth=2) Medium Term Notes (“MTN”) and Senior Notes
        - (ID 61, depth=2) Credit Facility
        - (ID 62, depth=2) Term Loan
        - (ID 63, depth=2) Mortgages Payable
        - (ID 64, depth=2) Dividends
        - (ID 65, depth=2) Hurricane Impact
        - (ID 66, depth=2) Income Taxes
    - (ID 67, depth=1) Contractual Obligations and Other Commitments
    - (ID 68, depth=1) Off-Balance Sheet Arrangements
        - (ID 69, depth=2) Unconsolidated Real Estate Joint Ventures
        - (ID 70, depth=2) Other Real Estate Investments
    - (ID 71, depth=1) Funds From Operations
    - (ID 72, depth=1) Same Property Net Operating Income (“Same property NOI”)
    - (ID 73, depth=1) Effects of Inflation
    - (ID 74, depth=1) New Accounting Pronouncements

In [26]:
tree_48 = HeadingTree("ITEM 7. MANAGEMENT’S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS", 58)
# Overview
tree_48.add_node(1, "Overview", 58, True, True, False)
# License Agreements and Business Relationships p59
tree_48.add_node(1, "License Agreements and Business Relationships", 59, False, True, False)
# Novartis
tree_48.add_node(2, "Novartis", 59, False, True, True)
# Lilly p60
tree_48.add_node(2, "Lilly", 60, False, True, True)
# Agenus p61
tree_48.add_node(2, "Agenus", 61, False, True, True)
# Takeda (ARIAD) p62
tree_48.add_node(2, "Takeda (ARIAD)", 62, False, True, True)
# Merus
tree_48.add_node(2, "Merus", 62, False, True, True)
# Calithera    p63
tree_48.add_node(2, "Calithera", 63, False, True, True)
# MacroGenics
tree_48.add_node(2, "MacroGenics", 63, False, True, True)
# Syros
tree_48.add_node(2, "Syros", 63, False, True, True)
# Innovent p64
tree_48.add_node(2, "Innovent", 64, False, True, True)
# Zai Lab
tree_48.add_node(2, "Zai Lab", 64, False, True, True)
# MorphoSys
tree_48.add_node(2, "MorphoSys", 64, False, True, True)
# Critical Accounting Policies and Significant Estimates - 65
tree_48.add_node(1, "Critical Accounting Policies and Significant Estimates", 65, False, True, False)
# Revenue Recognition
tree_48.add_node(2, "Revenue Recognition", 65, False, True, True)
# Product Revenues
tree_48.add_node(3, "Product Revenues", 65, False, False, True)
# Customer Credits
tree_48.add_node(4, "Customer Credits", 65, False, False, True)
# Rebates and Discounts     p66
tree_48.add_node(4, "Rebates and Discounts", 66, False, True, True)
# Chargebacks
tree_48.add_node(4, "Chargebacks", 66, False, True, True)
# Medicare Part D Coverage Gap
tree_48.add_node(4, "Medicare Part D Coverage Gap", 66, False, True, True)
# Co-payment Assistance
tree_48.add_node(4, "Co-payment Assistance", 66, False, True, True)
# Product Royalty Revenues
tree_48.add_node(3, "Product Royalty Revenues", 66, False, False, True)
# Milestone and Contract Revenues
tree_48.add_node(3, "Milestone and Contract Revenues", 66, False, False, True)
# Stock Compensation
tree_48.add_node(2, "Stock Compensation", 66, False, True, True)
# Income Taxes p67
tree_48.add_node(2, "Income Taxes", 67, False, True, True)
# Acquisition-related contingent consideration
tree_48.add_node(2, "Acquisition-related contingent consideration", 67, False, True, True)
# Results of Operations
tree_48.add_node(1, "Results of Operations", 67, False, True, False)
# Years Ended December 31, 2019 and 2018
tree_48.add_node(2, "Years Ended December 31, 2019 and 2018", 67, False, True, False)
# Revenues
tree_48.add_node(2, "Revenues", 67, False, True, False)
# Cost of Product Revenues p69
tree_48.add_node(2, "Cost of Product Revenues", 69, False, True, True)
# Operating Expenses p70
tree_48.add_node(2, "Operating Expenses", 70, False, True, True)
# Selling, general and administrative expenses
tree_48.add_node(3, "Selling, general and administrative expenses", 70, False, False, True)
# Change in fair value of acquisition-related contingent consideration   p71
tree_48.add_node(3, "Change in fair value of acquisition-related contingent consideration", 71, False, True, True)
# Other income (expense)
tree_48.add_node(2, "Other income (expense)", 71, False, True, True)
# Other income (expense), net
tree_48.add_node(3, "Other income (expense), net", 71, False, False, True)
# Interest expense
tree_48.add_node(3, "Interest expense", 71, False, False, True)
# Unrealized gain (loss) on long term investments
tree_48.add_node(3, "Unrealized gain (loss) on long term investments", 71, False, False, True)
# Provision for income taxes
tree_48.add_node(3, "Provision for income taxes", 71, False, True, True)
# Liquidity and Capital Resources p72
tree_48.add_node(1, "Liquidity and Capital Resources", 72, False, True, False)
# Sources and Uses of Cash.
tree_48.add_node(2, "Sources and Uses of Cash.", 72, False, True, False)
# Cash provided by operating activities
tree_48.add_node(3, "Cash provided by operating activities", 72, False, False, False)
# Cash used in investing activities
tree_48.add_node(3, "Cash used in investing activities", 72, False, False, False)
# Cash provided by financing activities
tree_48.add_node(3, "Cash provided by financing activities", 72, False, False, False)
# Off-Balance Sheet Arrangements p73
tree_48.add_node(1, "Off-Balance Sheet Arrangements", 73, False, True, False)

print(tree_48.show_tree_ascii())
tree_48.show_formatted_tree_in_notebook()




ITEM 7. MANAGEMENT’S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS
├── Overview
├── License Agreements and Business Relationships
│   ├── Novartis
│   ├── Lilly
│   ├── Agenus
│   ├── Takeda (ARIAD)
│   ├── Merus
│   ├── Calithera
│   ├── MacroGenics
│   ├── Syros
│   ├── Innovent
│   ├── Zai Lab
│   └── MorphoSys
├── Critical Accounting Policies and Significant Estimates
│   ├── Revenue Recognition
│   │   ├── Product Revenues
│   │   │   ├── Customer Credits
│   │   │   ├── Rebates and Discounts
│   │   │   ├── Chargebacks
│   │   │   ├── Medicare Part D Coverage Gap
│   │   │   └── Co-payment Assistance
│   │   ├── Product Royalty Revenues
│   │   └── Milestone and Contract Revenues
│   ├── Stock Compensation
│   ├── Income Taxes
│   └── Acquisition-related contingent consideration
├── Results of Operations
│   ├── Years Ended December 31, 2019 and 2018
│   ├── Revenues
│   ├── Cost of Product Revenues
│   ├── Operating Expenses
│   │   ├── Selling, genera

- (ID 0, depth=0) **ITEM 7. MANAGEMENT’S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS**
    - (ID 1, depth=1) **OVERVIEW**
    - (ID 2, depth=1) **License Agreements and Business Relationships**
        - (ID 3, depth=2) ***Novartis***
        - (ID 4, depth=2) ***Lilly***
        - (ID 5, depth=2) ***Agenus***
        - (ID 6, depth=2) ***Takeda (ARIAD)***
        - (ID 7, depth=2) ***Merus***
        - (ID 8, depth=2) ***Calithera***
        - (ID 9, depth=2) ***MacroGenics***
        - (ID 10, depth=2) ***Syros***
        - (ID 11, depth=2) ***Innovent***
        - (ID 12, depth=2) ***Zai Lab***
        - (ID 13, depth=2) ***MorphoSys***
    - (ID 14, depth=1) **Critical Accounting Policies and Significant Estimates**
        - (ID 15, depth=2) ***Revenue Recognition***
            - (ID 16, depth=3) *Product Revenues*
                - (ID 17, depth=4) *Customer Credits*
                - (ID 18, depth=4) ***Rebates and Discounts***
                - (ID 19, depth=4) ***Chargebacks***
                - (ID 20, depth=4) ***Medicare Part D Coverage Gap***
                - (ID 21, depth=4) ***Co-payment Assistance***
            - (ID 22, depth=3) *Product Royalty Revenues*
            - (ID 23, depth=3) *Milestone and Contract Revenues*
        - (ID 24, depth=2) ***Stock Compensation***
        - (ID 25, depth=2) ***Income Taxes***
        - (ID 26, depth=2) ***Acquisition-related contingent consideration***
    - (ID 27, depth=1) **Results of Operations**
        - (ID 28, depth=2) **Years Ended December 31, 2019 and 2018**
        - (ID 29, depth=2) **Revenues**
        - (ID 30, depth=2) ***Cost of Product Revenues***
        - (ID 31, depth=2) ***Operating Expenses***
            - (ID 32, depth=3) *Selling, general and administrative expenses*
            - (ID 33, depth=3) ***Change in fair value of acquisition-related contingent consideration***
        - (ID 34, depth=2) ***Other income (expense)***
            - (ID 35, depth=3) *Other income (expense), net*
            - (ID 36, depth=3) *Interest expense*
            - (ID 37, depth=3) *Unrealized gain (loss) on long term investments*
            - (ID 38, depth=3) ***Provision for income taxes***
    - (ID 39, depth=1) **Liquidity and Capital Resources**
        - (ID 40, depth=2) **Sources and Uses of Cash.**
            - (ID 41, depth=3) Cash provided by operating activities
            - (ID 42, depth=3) Cash used in investing activities
            - (ID 43, depth=3) Cash provided by financing activities
    - (ID 44, depth=1) **Off-Balance Sheet Arrangements**

In [29]:
tree_08 = HeadingTree("ITEM 1. BUSINESS", 3)
# Overview
tree_08.add_node(1, "Overview", 3, False, True, False)
# U.S. Information Solutions (USIS)
tree_08.add_node(2, "U.S. Information Solutions (USIS)", 3, False, True, False)
# International
tree_08.add_node(2, "International", 3, False, True, False)
#  Workforce Solutions
tree_08.add_node(2, "Workforce Solutions", 3, False, True, False)
# Global Consumer Solutions]
tree_08.add_node(2, "Global Consumer Solutions", 3, False, True, False)
# 2017 Cybersecurity Incident
tree_08.add_node(1, "2017 Cybersecurity Incident", 3, False, True, False)
# Our Business Strategy -4
tree_08.add_node(1, "Our Business Strategy", 4, False, True, False)
# Lead our industry in data security
tree_08.add_node(2, "Lead our industry in data security", 4, False, True, True)
# Transform our technology
tree_08.add_node(2, "Transform our technology", 4, False, True, True)
# Lead in data and analytics, to develop unparalleled analytical insights leveraging Equifax’s unique data
tree_08.add_node(2, "Lead in data and analytics, to develop unparalleled analytical insights leveraging Equifax’s unique data", 4, False, True, True)
# Improve the consumer user experience p5
tree_08.add_node(2, "Improve the consumer user experience", 5, False, True, True)
# Foster a culture of customer centricity
tree_08.add_node(2, "Foster a culture of customer centricity", 5, False, True, True)
# Deliver growth while enhancing profitability and shareholder returns
tree_08.add_node(2, "Deliver growth while enhancing profitability and shareholder returns", 5, False, True, True)
# Build a world-class Equifax team by investing in talent to drive our strategy and promote a culture of innovation
tree_08.add_node(2, "Build a world-class Equifax team by investing in talent to drive our strategy and promote a culture of innovation", 5, False, True, True)
# Markets and Clients p6
tree_08.add_node(1, "Markets and Clients", 6, False, True, False)
# Products and Services]
tree_08.add_node(1, "Products and Services", 6, False, True, False)
# USIS 07
tree_08.add_node(1, "USIS", 7, False, True, False)
# Online Information Solutions.
tree_08.add_node(2, "Online Information Solutions.", 7, False, True, True)
# Mortgage Solutions
tree_08.add_node(2, "Mortgage Solutions", 7, False, True, True)
# Financial Marketing Services
tree_08.add_node(2, "Financial Marketing Services", 7, False, True, True)
# International p8
tree_08.add_node(1, "International", 8, False, True, False)
# Europe
tree_08.add_node(2, "Europe", 8, False, True, True)
# Asia Pacific
tree_08.add_node(2, "Asia Pacific", 8, False, True, True)
# Latin America
tree_08.add_node(2, "Latin America", 8, False, True, True)
# Canada
tree_08.add_node(2, "Canada", 8, False, True, True)
# Workforce Solutions -9
tree_08.add_node(1, "Workforce Solutions", 9, False, True, False)
# Verification Services
tree_08.add_node(2, "Verification Services", 9, False, True, True)
# Employer Services
tree_08.add_node(2, "Employer Services", 9, False, True, True)
# The Work Number
tree_08.add_node(2, "The Work Number", 9, False, True, True)
# Global Consumer Solutions
tree_08.add_node(1, "Global Consumer Solutions", 9, False, True, False)
# Free Consumer Services
tree_08.add_node(1, "Free Consumer Services", 9, False, True, False)
# Seasonality p10
tree_08.add_node(1, "Seasonality", 10, False, True, False)
# Competition
tree_08.add_node(1, "Competition", 10, False, True, False)
# Technology and Intellectual Property p11
tree_08.add_node(1, "Technology and Intellectual Property", 11, False, True, False)
# Governmental Regulation 
tree_08.add_node(1, "Governmental Regulation", 11, False, True, False)
# Summary of U.S. Regulation Relating to Consumer and Data Protection
tree_08.add_node(2, "Summary of U.S. Regulation Relating to Consumer and Data Protection", 11, False, True, True)
# Federal Laws and Regulation p12
tree_08.add_node(3, "Federal Laws and Regulation", 12, False, False, True)
# FCRA 
tree_08.add_node(4, "FCRA", 12, False, False, False)
# The Dodd-Frank Act
tree_08.add_node(4, "The Dodd-Frank Act", 12, False, False, False)
# FTC Act
tree_08.add_node(4, "FTC Act", 12, False, False, False)
# GLBA
tree_08.add_node(4, "GLBA", 12, False, False, False)
# CROA
tree_08.add_node(4, "CROA", 12, False, False, False)
# State Laws and Regulation Relating to Consumer and Data Protection p13
tree_08.add_node(3, "State Laws and Regulation Relating to Consumer and Data Protection", 13, False, False, True)
# Summary of International Regulation Relating to Consumer and Data Protection
tree_08.add_node(2, "Summary of International Regulation Relating to Consumer and Data Protection", 13, False, True, True)
# In the U.K
tree_08.add_node(3, "In the U.K", 13, False, True, False)
# In Europe
tree_08.add_node(3, "In Europe", 13, False, True, False)
# In canada p14
tree_08.add_node(3, "In canada", 14, False, True, True)
# In Latin America
tree_08.add_node(3, "In Latin America", 14, False, True, False)
# In Australia
tree_08.add_node(3, "In Australia", 14, False, True, False)
# In New Zealand
tree_08.add_node(3, "In New Zealand", 14, False, True, False)
# In India p15
tree_08.add_node(3, "In India", 15, False, True, True)
# In Russia
tree_08.add_node(3, "In Russia", 15, False, True, False)
# Tax Management Services
tree_08.add_node(2, "Tax Management Services", 15, False, True, True)
# Personnel
tree_08.add_node(1, "Personnel", 15, False, True, False)
# Foward-Looking Statements 
tree_08.add_node(1, "Foward-Looking Statements", 15, False, True, False)
# Available Information 
tree_08.add_node(1, "Available Information", 15, False, True, False)

print(tree_08.show_tree_ascii())
tree_08.show_formatted_tree_in_notebook()




ITEM 1. BUSINESS
├── Overview
│   ├── U.S. Information Solutions (USIS)
│   ├── International
│   ├── Workforce Solutions
│   └── Global Consumer Solutions
├── 2017 Cybersecurity Incident
├── Our Business Strategy
│   ├── Lead our industry in data security
│   ├── Transform our technology
│   ├── Lead in data and analytics, to develop unparalleled analytical insights leveraging Equifax’s unique data
│   ├── Improve the consumer user experience
│   ├── Foster a culture of customer centricity
│   ├── Deliver growth while enhancing profitability and shareholder returns
│   └── Build a world-class Equifax team by investing in talent to drive our strategy and promote a culture of innovation
├── Markets and Clients
├── Products and Services
├── USIS
│   ├── Online Information Solutions.
│   ├── Mortgage Solutions
│   └── Financial Marketing Services
├── International
│   ├── Europe
│   ├── Asia Pacific
│   ├── Latin America
│   └── Canada
├── Workforce Solutions
│   ├── Verification Services

- (ID 0, depth=0) **ITEM 1. BUSINESS**
    - (ID 1, depth=1) **Overview**
        - (ID 2, depth=2) **U.S. Information Solutions (USIS)**
        - (ID 3, depth=2) **International**
        - (ID 4, depth=2) **Workforce Solutions**
        - (ID 5, depth=2) **Global Consumer Solutions**
    - (ID 6, depth=1) **2017 Cybersecurity Incident**
    - (ID 7, depth=1) **Our Business Strategy**
        - (ID 8, depth=2) ***Lead our industry in data security***
        - (ID 9, depth=2) ***Transform our technology***
        - (ID 10, depth=2) ***Lead in data and analytics, to develop unparalleled analytical insights leveraging Equifax’s unique data***
        - (ID 11, depth=2) ***Improve the consumer user experience***
        - (ID 12, depth=2) ***Foster a culture of customer centricity***
        - (ID 13, depth=2) ***Deliver growth while enhancing profitability and shareholder returns***
        - (ID 14, depth=2) ***Build a world-class Equifax team by investing in talent to drive our strategy and promote a culture of innovation***
    - (ID 15, depth=1) **Markets and Clients**
    - (ID 16, depth=1) **Products and Services**
    - (ID 17, depth=1) **USIS**
        - (ID 18, depth=2) ***Online Information Solutions.***
        - (ID 19, depth=2) ***Mortgage Solutions***
        - (ID 20, depth=2) ***Financial Marketing Services***
    - (ID 21, depth=1) **International**
        - (ID 22, depth=2) ***Europe***
        - (ID 23, depth=2) ***Asia Pacific***
        - (ID 24, depth=2) ***Latin America***
        - (ID 25, depth=2) ***Canada***
    - (ID 26, depth=1) **Workforce Solutions**
        - (ID 27, depth=2) ***Verification Services***
        - (ID 28, depth=2) ***Employer Services***
        - (ID 29, depth=2) ***The Work Number***
    - (ID 30, depth=1) **Global Consumer Solutions**
    - (ID 31, depth=1) **Free Consumer Services**
    - (ID 32, depth=1) **Seasonality**
    - (ID 33, depth=1) **Competition**
    - (ID 34, depth=1) **Technology and Intellectual Property**
    - (ID 35, depth=1) **Governmental Regulation**
        - (ID 36, depth=2) ***Summary of U.S. Regulation Relating to Consumer and Data Protection***
            - (ID 37, depth=3) *Federal Laws and Regulation*
                - (ID 38, depth=4) FCRA
                - (ID 39, depth=4) The Dodd-Frank Act
                - (ID 40, depth=4) FTC Act
                - (ID 41, depth=4) GLBA
                - (ID 42, depth=4) CROA
            - (ID 43, depth=3) *State Laws and Regulation Relating to Consumer and Data Protection*
        - (ID 44, depth=2) ***Summary of International Regulation Relating to Consumer and Data Protection***
            - (ID 45, depth=3) **In the U.K**
            - (ID 46, depth=3) **In Europe**
            - (ID 47, depth=3) ***In canada***
            - (ID 48, depth=3) **In Latin America**
            - (ID 49, depth=3) **In Australia**
            - (ID 50, depth=3) **In New Zealand**
            - (ID 51, depth=3) ***In India***
            - (ID 52, depth=3) **In Russia**
        - (ID 53, depth=2) ***Tax Management Services***
    - (ID 54, depth=1) **Personnel**
    - (ID 55, depth=1) **Foward-Looking Statements**
    - (ID 56, depth=1) **Available Information**